In [ ]:
# wide and embedding, 
# wide co-occurrence
# wide co-occurrence & interaction 

# TO DO
# batch normalization
# weight normalization
# max norm constraint: dropout paper 

# start with low regularization, large loss
#  more parameters, high learning rate


In [1]:
import gzip
import os
import tempfile

import numpy as np
import pandas as pd
# from six.moves import urllib
# from six.moves import xrange  # pylint: disable=redefined-builtin


import tensorflow as tf
from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.examples.tutorials.mnist import mnist
from tensorflow.contrib import rnn


import math
import random
import time
import datetime

from keras.utils import np_utils

from sklearn import preprocessing



Couldn't import dot_parser, loading of dot files will not be possible.


Using Theano backend.


In [2]:
# continuous and categorical feature split

# features on columns [0:split_idx] are continuous
# the rest are categorical
def conti_cate_split(dta_df, split_idx):
    
    col_cnt= dta_df.shape[1]
    conti_cols= range(split_idx)
    dis_cols=range(split_idx, col_cnt)
    
    return dta_df[conti_cols], dta_df[dis_cols]

def conti_normalization_train_dta(dta_df):
    
    return preprocessing.scale(dta_df)

def conti_normalization_test_dta(dta_df, train_df):
    
    mean_dim = np.mean(train_df, axis=0)
    std_dim = np.std(train_df, axis=0)
        
    df=pd.DataFrame()
    cols = train_df.columns
    idx=0
    
    for i in cols:
        df[i] = (dta_df[i]- mean_dim[idx])*1.0/std_dim[idx]
        idx=idx+1
        
    return df


# ------------data prepro-----------------------

files_list=["../dataset/dataset_demo/xtrain.csv", \
            "../dataset/dataset_demo/xtest.csv",\
            "../dataset/dataset_demo/ytrain.csv", \
            "../dataset/dataset_demo/ytest.csv"]

xtrain_df=pd.read_csv( files_list[0] ,sep=',', header=None)
xtest_df=pd.read_csv( files_list[1] ,sep=',', header=None)
ytrain_df=pd.read_csv( files_list[2] ,sep=',', header=None)
ytest_df=pd.read_csv( files_list[3] ,sep=',', header=None)

print xtrain_df.shape, xtest_df.shape, ytrain_df.shape, ytest_df.shape


# get training and testing data prepared

cxtrain, dxtrain = conti_cate_split(xtrain_df, 106)
cxtest, dxtest = conti_cate_split(xtest_df, 106)

cxtest = conti_normalization_test_dta(cxtest, cxtrain)
cxtrain = conti_normalization_train_dta(cxtrain)

print cxtrain.shape, dxtrain.shape
print cxtest.shape, dxtest.shape

# cxtrain = cxtrain.as_matrix() 
dxtrain = dxtrain.as_matrix().astype(int)
cxtest = cxtest.as_matrix()
dxtest = dxtest.as_matrix().astype(int)

ytest = np_utils.to_categorical( ytest_df.as_matrix() )
ytrain =np_utils.to_categorical( ytrain_df.as_matrix() )

(6938, 186) (771, 186) (6938, 1) (771, 1)
(6938, 106) (6938, 80)
(771, 106) (771, 80)


In [4]:
# wide, embedding, co-occurrence and feature interaction 
# version 1: interaction as external hiddens

class wide_embed_coocc_NN():
    
#   build the network graph
    def __init__(self, n_conti, n_disc, n_class,\
                 n_disc_voca, session,n_embedding, n_hidden_list, lr, l2):
        
        
        self.LEARNING_RATE = lr
                
        self.N_CLASS = n_class
        self.N_CONTI = n_conti
        self.N_DISC = n_disc
        self.N_DISC_VOCA = n_disc_voca
        self.N_EMBED = n_embedding
        self.L2 = l2
        
        self.N_HIDDEN_LAYERS = len(n_hidden_list)
        self.N_TOTAL = self.N_CONTI+ self.N_DISC*self.N_EMBED

        self.sess=session
        
        self.dx = tf.placeholder(tf.int32, [None, self.N_DISC])
        self.cx = tf.placeholder(tf.float32, [None, self.N_CONTI])
        self.y = tf.placeholder(tf.float32, [None, self.N_CLASS])
        self.lr = tf.placeholder(tf.float32, shape=())
        self.keep_prob = tf.placeholder(tf.float32)
        
#       wide part on categorical features
        self.dx_trans = tf.transpose( self.dx, [1,0] )
        
#         for i in range(self.N_DISC):
            
#             with tf.variable_scope("wide"+str(i)):
#                 w= tf.Variable(\
#                 tf.random_uniform([self.N_DISC_VOCA[i], self.N_CLASS],-1.0, 1.0) )
# #                         tf.random_normal([self.N_DISC_VOCA, self.N_CLASS],\
# #                         stddev=math.sqrt(2.0/float(self.N_DISC_VOCA))) )

#                 b= tf.Variable(tf.zeros( [ self.N_CLASS ] ))

#                 if i==0:
#                     dx_wsum = tf.nn.embedding_lookup( w, self.dx_trans[i] ) + b
                    
# #                   L2
#                     self.regularizer_wide = tf.nn.l2_loss(w)
                    
#                 else:
#                     dx_wsum = dx_wsum + tf.nn.embedding_lookup( w, self.dx_trans[i] ) + b
                    
# #                   L2  
#                     self.regularizer_wide = self.regularizer_wide + tf.nn.l2_loss(w)
        
        
# #       non-linear
#         dx_wsum = tf.nn.relu( dx_wsum )

        
#       co-occurrence of categorical features
        for i in range(20):
            
            tmpvoca1 = self.N_DISC_VOCA[i] 
            
            for j in range(i+1,20):
                
                tmpvoca2 = self.N_DISC_VOCA[j]
                
                with tf.variable_scope("cooccur"+str(i)+str(j)):
                    
#                   vector approximate of co-occurrence matrix 
                    w1 = tf.Variable(tf.random_normal([self.N_CLASS, tmpvoca1, 1],\
                                            stddev=math.sqrt(2.0/float( tmpvoca1 ))) )
                    
                    w2 = tf.Variable(tf.random_normal([self.N_CLASS, 1, tmpvoca2],\
                                            stddev=math.sqrt(2.0/float( tmpvoca2 ))) )
            
                    
                    b= tf.Variable(tf.zeros( [ self.N_CLASS ] ))

                    
                    cooc_mat = []
                    for k in range(self.N_CLASS):
                        cooc_mat.append( tf.reshape( tf.matmul(w1[k],w2[k]), [-1,1] ) )
                        
#                   n_class by voca1*voca2  
                    cooc_vec = tf.stack(cooc_mat)
                    cooc_vec = tf.reshape(cooc_vec, [self.N_CLASS, -1] )
                    cooc_vec = tf.transpose(cooc_vec,[1,0])
                    
#                   build the idx of co-occurrence features
                    cooc_idx = self.dx_trans[i] * tmpvoca2 + self.dx_trans[j]
    
                    if i==0 and j==1:
                        cooc_wsum  = tf.nn.embedding_lookup( cooc_vec, cooc_idx ) + b
#                       L2
                        self.regularizer_cooc   = tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2)
                    else:
                        cooc_wsum += tf.nn.embedding_lookup( cooc_vec, cooc_idx ) + b
#                       L2
                        self.regularizer_cooc += ( tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2) )
    
        
#       non-linear
        cooc_wsum = tf.nn.relu( cooc_wsum )

    
#       feature interaction
        for i in range(self.N_DISC):
            
            with tf.variable_scope("interact"+str(i)):
                w= tf.Variable(\
                        tf.random_normal([self.N_DISC_VOCA[i], self.N_CONTI],\
                        stddev=math.sqrt(2.0/float(self.N_DISC_VOCA[i]))) )
#                     tf.random_uniform([self.N_DISC_VOCA, self.N_EMBED],-1.0, 1.0 )
        
                if i==0:
                    inter_wsum  = tf.nn.embedding_lookup( w, self.dx_trans[i] )
#                   L2 
                    self.regularizer_inter  = tf.nn.l2_loss(w)
            
                else:
                    inter_wsum += tf.nn.embedding_lookup( w, self.dx_trans[i] )
#                   L2 
                    self.regularizer_inter += tf.nn.l2_loss(w)

    
#       add noise bias
        singleCol_h = tf.reduce_sum( self.cx * inter_wsum, 1 )
        singleCol_h = tf.reshape( singleCol_h, [-1,1] )
        
#         tmp_h=[]
#         for i in range( n_hidden_list[0] ):
#             with tf.variable_scope("noise"+str( i )):
#                 noise_b = tf.Variable(tf.random_normal([ ],\
#                         stddev=math.sqrt(2.0/float(n_hidden_list[0]))))
#                 tmp_h.append( singleCol_h + noise_b ) 
        
#         tmp_h = tf.stack( tmp_h )
#         tmp_h = tf.transpose( tmp_h, [1,0])
            
        
#       interaction hidden layers
        with tf.variable_scope("inter_h0"):
            w = tf.Variable(tf.random_normal([1, n_hidden_list[0]],\
                        stddev=math.sqrt(2.0/float(n_hidden_list[0])))) 
                        
            b = tf.Variable(tf.zeros( [ n_hidden_list[0] ] ))
            
            h_inter = tf.matmul( singleCol_h, w )
#             h_inter = h_inter + tmp_h
            h_inter = tf.nn.relu( h_inter + b)
#           L2  
            self.regularizer_inter += tf.nn.l2_loss(w)
            
        for i in range(1, self.N_HIDDEN_LAYERS):
            
            with tf.variable_scope("inter_h"+str(i)):
                w = tf.Variable(\
                        tf.random_normal([n_hidden_list[i-1],n_hidden_list[i]],\
                                stddev=math.sqrt(2.0/float(n_hidden_list[i-1])))) 
                b = tf.Variable(tf.zeros( [n_hidden_list[i]] ))
                h_inter = tf.nn.relu( tf.add( tf.matmul(h_inter, w),b) )
#               L2
                self.regularizer_inter += tf.nn.l2_loss(w)
    
    
#       embedding of categorical features        
        tmp_embeded = []                
        for i in range(self.N_DISC):
            
            with tf.variable_scope("embedding"+str(i)):
                w= tf.Variable(\
                        tf.random_normal([self.N_DISC_VOCA[i], self.N_EMBED],\
                        stddev=math.sqrt(2.0/float(self.N_DISC_VOCA[i]))) )
#                     tf.random_uniform([self.N_DISC_VOCA, self.N_EMBED],-1.0, 1.0 )
                              
                tmp_embeded.append( tf.nn.embedding_lookup( w, self.dx_trans[i] ) )
                
        dx_embeded = tf.concat(tmp_embeded, 1)
        x_concate =  tf.concat( [self.cx,dx_embeded], 1 )
        
#       embedding + continuous hidden layers
        with tf.variable_scope("h0"):
            w= tf.Variable(tf.random_normal([self.N_TOTAL, n_hidden_list[0]],\
                        stddev=math.sqrt(2.0/float(self.N_CONTI)))) 
            b= tf.Variable(tf.zeros( [ n_hidden_list[0] ] ))
            h = tf.nn.relu( tf.add( tf.matmul(x_concate, w),b) )
#           L2
            self.regularizer = tf.nn.l2_loss(w)
        
                
        for i in range(1, self.N_HIDDEN_LAYERS):
            
            with tf.variable_scope("layer"+str(i)):
                w= tf.Variable(\
                        tf.random_normal([n_hidden_list[i-1],n_hidden_list[i]],\
                                stddev=math.sqrt(2.0/float(n_hidden_list[i-1])))) 
                b= tf.Variable(tf.zeros( [n_hidden_list[i]] ))
                h = tf.nn.relu( tf.add( tf.matmul(h, w),b) )
                
#               L2
                self.regularizer += tf.nn.l2_loss(w)
        
                
#       dropout
#       h = tf.nn.dropout(h, self.keep_prob)

        
#       output layer  
        h = tf.concat([h, h_inter], 1)
    
        with tf.variable_scope("temp"):
            w= tf.Variable(tf.random_normal([n_hidden_list[self.N_HIDDEN_LAYERS-1]*2,\
                                             n_hidden_list[self.N_HIDDEN_LAYERS-1] ],\
                        stddev=math.sqrt(2.0/float(n_hidden_list[self.N_HIDDEN_LAYERS-1]))))
                           
            b= tf.Variable(tf.zeros( [ n_hidden_list[self.N_HIDDEN_LAYERS-1] ] ))
            h = tf.nn.relu( tf.add( tf.matmul(h, w),b) )
                           
#           L2  
            self.regularizer += tf.nn.l2_loss(w) 

    
        with tf.variable_scope("output"):
            
            w= tf.Variable(tf.random_normal([n_hidden_list[self.N_HIDDEN_LAYERS-1],\
                                             self.N_CLASS],\
            stddev=math.sqrt(2.0/float(n_hidden_list[self.N_HIDDEN_LAYERS-1])))) 
            b= tf.Variable(tf.zeros( [ self.N_CLASS ] ))
            
            h = tf.matmul(h, w)
#             h = tf.add( h, dx_wsum ) #wide
            h = tf.add( h, cooc_wsum ) #co-occurrence
            h = tf.add( h, b )

            self.logit = h
     
    
#           L2  
            self.regularizer += tf.nn.l2_loss(w) 
    
#       overall L2
        self.regularizer += (self.regularizer_cooc + self.regularizer_inter)
#     self.regularizer_wide + 
         
    
#   initialize loss and optimization operations for training
    def train_ini(self):
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(\
                                                    logits=self.logit, labels=self.y) \
                                  + self.L2*self.regularizer )
        self.optimizer = \
        tf.train.AdadeltaOptimizer(learning_rate = self.LEARNING_RATE).minimize(self.cost)
#       tf.train.RMSPropOptimizer(learning_rate = self.LEARNING_RATE).minimize(self.cost)
#         tf.train.AdamOptimizer(learning_rate = self.lr).minimize(self.cost)
#         tf.train.RMSPropOptimizer(learning_rate = self.lr).minimize(self.cost)
#         tf.train.AdadeltaOptimizer(learning_rate = self.lr).minimize(self.cost)
#         tf.train.AdamOptimizer(learning_rate = self.lr).minimize(self.cost)
#         tf.train.GradientDescentOptimizer(learning_rate = self.lr).minimize(self.cost)
        
        self.init = tf.global_variables_initializer()
        sess.run( self.init )
        
#   training on batch of data
    def train_batch(self, dx_batch,cx_batch, y_batch, keep_prob, lr):
        
        _,c = sess.run([self.optimizer,self.cost],\
                       feed_dict={self.dx:dx_batch, \
                                  self.cx:cx_batch, self.y:y_batch,\
                                  self.lr:lr,\
                                  self.keep_prob:keep_prob\
                                 })
        
        return c
    
#   initialize inference         
    def inference_ini(self):
        self.correct_prediction = tf.equal(tf.argmax(self.logit,1), tf.argmax(self.y,1))
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, "float"))
        
#   infer givn testing data    
    def inference(self, dx_test, cx_test, y_test, keep_prob):
        return sess.run([self.accuracy], feed_dict={self.dx:dx_test,\
                                                  self.cx:cx_test, self.y:y_test,\
                                                  self.keep_prob:keep_prob\
                                                   })
        
#   unit_test
    def test(self, dx_test, cx_test, y_test ):
        self.init = tf.global_variables_initializer()
        sess.run( self.init )
        return sess.run( [self.tmpshape], \
                 feed_dict={self.dx:dx_test, self.cx:cx_test, self.y:y_test})


In [6]:
# wide, embedding, co-occurrence and feature interaction 
# version 1: interaction as external hiddens

para_n_epoch = 1200

# tunable parameters

#   representation ability
para_n_hidden_list = [ 64,32,16 ]
#     128,16,8 ]
para_lr = 0.045
para_n_embedding = 2
#   regularization
para_batch_size = 32
para_keep_prob = 1.0
para_l2 = 0.1

# fixed parameters
para_n_class = 8
para_n_disc = 80
para_n_conti = 106
para_n_disc_voca = [8]*para_n_disc

# evaluation parameters
para_eval_byepoch = 10


with tf.Session() as sess:
    
    clf = wide_embed_coocc_NN( para_n_conti, para_n_disc, para_n_class, \
                      para_n_disc_voca, sess, para_n_embedding, para_n_hidden_list,\
                        para_lr, para_l2)
    clf.train_ini()
    clf.inference_ini()
    
    para_cur_lr=para_lr
    
    total_cnt= np.shape(cxtrain)[0]
    total_batch = int(total_cnt/para_batch_size)
    
    total_idx=range(total_cnt)
    
    
#   training cycle
    for epoch in range(para_n_epoch):
        
        tmpc=0.0
        
#       shuffle traning instances each epoch  
        np.random.shuffle(total_idx)
        
#       Loop over all batches
        for i in range(total_batch):
            
            batch_idx = total_idx[ i*para_batch_size: (i+1)*para_batch_size ] 
            
            batch_cx = cxtrain[ batch_idx ]
            batch_dx = dxtrain[ batch_idx ]
            batch_y  =  ytrain[ batch_idx ]
            
            tmpc += clf.train_batch( batch_dx, batch_cx, batch_y, para_keep_prob,\
                                     para_cur_lr )
        
        if epoch%para_eval_byepoch != 0:
            continue
        
        tmp_test_acc  = clf.inference(dxtest, cxtest, ytest, para_keep_prob) 
        tmp_train_acc = clf.inference(dxtrain, cxtrain, ytrain, para_keep_prob) 
        
        print "loss on epoch ", epoch, " : ", 1.0*tmpc/total_batch, tmp_test_acc,\
        tmp_train_acc
        
    print "Optimization Finished!"

loss on epoch  0  :  2397.61354348 [0.23216602] [0.20942634]
loss on epoch  10  :  1609.22220922 [0.2542153] [0.24517152]
loss on epoch  20  :  890.686493202 [0.25032425] [0.24416259]
loss on epoch  30  :  428.261118571 [0.25551233] [0.24589218]
loss on epoch  40  :  179.341532813 [0.25551233] [0.24589218]
loss on epoch  50  :  65.1177196149 [0.25551233] [0.24589218]
loss on epoch  60  :  20.8747887965 [0.25551233] [0.24589218]
loss on epoch  70  :  6.55449533904 [0.25551233] [0.24589218]
loss on epoch  80  :  2.73966475549 [0.25551233] [0.24589218]
loss on epoch  90  :  1.92791347316 [0.25551233] [0.24589218]
loss on epoch  100  :  1.79373374416 [0.25551233] [0.24589218]


KeyboardInterrupt: 

In [ ]:
[ 64,32,16 ]

0.055 0.25 

0.055 0.15

0.065 0.1 all regu

loss on epoch  0  :  1172.07503877 [0.16601816] [0.17036609]
loss on epoch  10  :  800.301286203 [0.21400778] [0.23205535]
loss on epoch  20  :  448.827867861 [0.25162128] [0.26722398]
loss on epoch  30  :  216.808312875 [0.2697795] [0.26635918]
loss on epoch  40  :  91.3842687254 [0.29182878] [0.27428654]
loss on epoch  50  :  34.0733124768 [0.30609599] [0.28365523]
loss on epoch  60  :  11.7855009282 [0.31776914] [0.28740272]
loss on epoch  70  :  4.48057140907 [0.28534371] [0.26289997]
loss on epoch  80  :  2.49305601584 [0.28145266] [0.26592678]
loss on epoch  90  :  2.02818673408 [0.2542153] [0.24603632]
loss on epoch  100  :  1.91188755576 [0.25551233] [0.24589218]
loss on epoch  110  :  1.86578034582 [0.25551233] [0.24589218]
loss on epoch  120  :  1.83540236564 [0.25551233] [0.24589218]
loss on epoch  130  :  1.81479061312 [0.25551233] [0.24589218]
loss on epoch  140  :  1.79988374147 [0.25551233] [0.24589218]
loss on epoch  150  :  1.78881633227 [0.25551233] [0.24589218]
loss on epoch  160  :  1.7794387186 [0.25551233] [0.24589218]
loss on epoch  170  :  1.77178843706 [0.25551233] [0.24589218]
loss on epoch  180  :  1.76583942274 [0.25551233] [0.24589218]
loss on epoch  190  :  1.76063220037 [0.25551233] [0.24589218]
loss on epoch  200  :  1.75678650704 [0.25551233] [0.24589218]
loss on epoch  210  :  1.75275016493 [0.25551233] [0.24589218]
loss on epoch  220  :  1.75108405727 [0.25551233] [0.24589218]
loss on epoch  230  :  1.74886897206 [0.25551233] [0.24589218]
loss on epoch  240  :  1.74611885459 [0.25551233] [0.24589218]
loss on epoch  250  :  1.74387550133 [0.25551233] [0.24589218]
loss on epoch 

0.065 0.1 drop some L2, add interaction regu

loss on epoch  0  :  1167.35836001 [0.25810635] [0.23565869]
loss on epoch  10  :  799.897740117 [0.23476005] [0.24257711]
loss on epoch  20  :  449.965487021 [0.25551233] [0.26434129]
loss on epoch  30  :  218.084557498 [0.25551233] [0.26837707]
loss on epoch  40  :  92.3575367398 [0.26329443] [0.28567311]
loss on epoch  50  :  34.6127185292 [0.28793773] [0.30037475]
loss on epoch  60  :  12.138738981 [0.30998704] [0.31522053]
loss on epoch  70  :  4.73840802025 [0.32166019] [0.3248775]
loss on epoch  80  :  2.6576786847 [0.32814527] [0.33439031]
loss on epoch  90  :  2.11652825442 [0.30479896] [0.3441914]
loss on epoch  100  :  1.94270474105 [0.30220494] [0.35745171]
loss on epoch  110  :  1.85257242344 [0.31128404] [0.37705392]
loss on epoch  120  :  1.77273416464 [0.31128404] [0.41582587]
loss on epoch  130  :  1.63825527275 [0.28664073] [0.49827039]
loss on epoch  140  :  1.46095741568 [0.28145266] [0.58835399]
loss on epoch  150  :  1.29705251036 [0.27237353] [0.64917845]
loss on epoch  160  :  1.16783825032 [0.26718548] [0.68881524]
loss on epoch  170  :  1.06029736609 [0.25940338] [0.72023636]
loss on epoch  180  :  0.966977777029 [0.25291827] [0.79633898]


0.065 0.1 drop some L2

loss on epoch  0  :  325.844520993 [0.14396887] [0.13116172]
loss on epoch  10  :  217.151938403 [0.21530479] [0.2267224]
loss on epoch  20  :  122.24666433 [0.28015563] [0.27990776]
loss on epoch  30  :  59.594521081 [0.28534371] [0.29187086]
loss on epoch  40  :  25.9309680992 [0.30609599] [0.294177]
loss on epoch  50  :  10.6316536621 [0.29831389] [0.31320265]
loss on epoch  60  :  4.76818786506 [0.32295719] [0.31781495]
loss on epoch  70  :  2.88817745889 [0.32295719] [0.32819256]
loss on epoch  80  :  2.37351186849 [0.30090791] [0.3377054]
loss on epoch  90  :  2.23753988357 [0.30479896] [0.34087634]
loss on epoch  100  :  2.18417358785 [0.31647211] [0.34202939]
loss on epoch  110  :  2.14660033253 [0.32555124] [0.34202939]
loss on epoch  120  :  2.11423155997 [0.31776914] [0.34851542]
loss on epoch  130  :  2.08518664539 [0.31776914] [0.35038915]
loss on epoch  140  :  2.0599590264 [0.30609599] [0.3501009]
loss on epoch  150  :  2.03664129255 [0.30869001] [0.35168636]
loss on epoch  160  :  2.01500192505 [0.31647211] [0.35673106]
loss on epoch  170  :  1.99557032629 [0.31258106] [0.35946959]
loss on epoch  180  :  1.97540657222 [0.32036316] [0.36321706]
loss on epoch  190  :  1.95726226049 [0.31128404] [0.36422601]
loss on epoch  200  :  1.94178240277 [0.30609599] [0.36364946]
loss on epoch  210  :  1.92472366713 [0.30998704] [0.3735947]
loss on epoch 

0.065 0.1

loss on epoch  0  :  332.805889836 [0.17509727] [0.1836264]
loss on epoch  10  :  224.487996984 [0.23605707] [0.23882963]
loss on epoch  20  :  126.475320039 [0.27885863] [0.2711156]
loss on epoch  30  :  61.8446098963 [0.29961088] [0.28596136]
loss on epoch  40  :  27.0856315825 [0.29831389] [0.2822139]
loss on epoch  50  :  11.1628189175 [0.28793773] [0.27241281]
loss on epoch  60  :  4.88656196329 [0.28534371] [0.27558374]
loss on epoch  70  :  2.78419123314 [0.28664073] [0.27399826]
loss on epoch  80  :  2.17331035546 [0.26070037] [0.27053908]
loss on epoch  90  :  1.99862699707 [0.24513619] [0.26347652]
loss on epoch  100  :  1.93286249318 [0.23735408] [0.26347652]
loss on epoch  110  :  1.89984143884 [0.2386511] [0.26390889]
loss on epoch  120  :  1.87880094129 [0.23994812] [0.26578265]
loss on epoch  130  :  1.86088351243 [0.24254215] [0.27212453]
loss on epoch  140  :  1.84647996613 [0.2464332] [0.27471894]
loss on epoch  150  :  1.83207054105 [0.24773023] [0.27644855]
loss on epoch  160  :  1.81919553048 [0.25291827] [0.27745748]
loss on epoch  170  :  1.80824454515 [0.25162128] [0.28062841]
loss on epoch  180  :  1.79873686643 [0.24902724] [0.28077257]
loss on epoch  190  :  1.78950325427 [0.25032425] [0.2835111]
loss on epoch  200  :  1.78142097427 [0.24773023] [0.28495243]
loss on epoch  210  :  1.77397503069 [0.25032425] [0.28552896]
loss on epoch  220  :  1.76741607322 [0.24383917] [0.28466415]
loss on epoch  230  :  1.76024768419 [0.24124514] [0.28740272]
loss on epoch  240  :  1.75431586029 [0.23994812] [0.28942057]
loss on epoch  250  :  1.74874272115 [0.22827497] [0.28869992]
loss on epoch  260  :  1.74332360978 [0.2386511] [0.29172671]
loss on epoch  270  :  1.73768643373 [0.24254215] [0.29662728]
loss on epoch  280  :  1.73303826153 [0.23994812] [0.29749209]
loss on epoch  290  :  1.72819698795 [0.23605707] [0.29388872]
loss on epoch  300  :  1.72463215667 [0.23994812] [0.29864514]
loss on epoch  310  :  1.72031790846 [0.23735408] [0.30354568]
loss on epoch  320  :  1.71613417952 [0.2386511] [0.30455464]
loss on epoch  330  :  1.71238634542 [0.23735408] [0.30426636]
loss on epoch  340  :  1.7087495774 [0.23994812] [0.30368984]
loss on epoch  350  :  1.70528704645 [0.24254215] [0.30815798]
loss on epoch  360  :  1.70067572925 [0.23994812] [0.30599597]
loss on epoch  370  :  1.69772587882 [0.24124514] [0.31147304]
loss on epoch  380  :  1.69505295157 [0.24383917] [0.31132892]
loss on epoch  390  :  1.69087570409 [0.2464332] [0.30974343]
loss on epoch  400  :  1.68872178594 [0.24513619] [0.31132892


In [3]:
# wide, embedding, co-occurrence and feature interaction 
# version 2: individual hidden layers on embedding and interaction

class InterNN_IndiH():
    
    def __init__(self, n_conti, n_disc, n_class,\
                 n_disc_voca, session,n_embedding, n_hidden_list, lr, l2, batch_size):
        
#       build the network graph 
        self.LEARNING_RATE = lr
                
        self.N_CLASS = n_class
        self.N_CONTI = n_conti
        self.N_DISC = n_disc
        self.N_DISC_VOCA = n_disc_voca
        self.N_EMBED = n_embedding
        self.N_BATCH = batch_size
        self.L2 = l2
        
        self.N_HIDDEN_LAYERS = len(n_hidden_list)
        self.N_TOTAL = self.N_CONTI+ self.N_DISC*self.N_EMBED
        self.N_TOTAL_EMBED = self.N_DISC*self.N_EMBED

        self.sess=session
        
        self.dx = tf.placeholder(tf.int32, [None, self.N_DISC])
        self.cx = tf.placeholder(tf.float32, [None, self.N_CONTI])
        self.y = tf.placeholder(tf.float32, [None, self.N_CLASS])
        self.lr = tf.placeholder(tf.float32, shape=())
        self.keep_prob = tf.placeholder(tf.float32)
        
#       wide part on categorical features
        self.dx_trans = tf.transpose( self.dx, [1,0] )
    
    
    
#         for i in range(self.N_DISC):
            
#             with tf.variable_scope("wide"+str(i)):
#                 w= tf.Variable(\
#                 tf.random_uniform([self.N_DISC_VOCA[i], self.N_CLASS],-1.0, 1.0) )
# #                         tf.random_normal([self.N_DISC_VOCA, self.N_CLASS],\
# #                         stddev=math.sqrt(2.0/float(self.N_DISC_VOCA))) )
#                 if i==0:
#                     dx_wsum = tf.nn.embedding_lookup( w, self.dx_trans[i] )
                    
# #                   L2
#                     self.regularizer_wide = tf.nn.l2_loss(w)
                    
#                 else:
#                     dx_wsum = dx_wsum + tf.nn.embedding_lookup( w, self.dx_trans[i] )
                    
# #                   L2  
#                     self.regularizer_wide = self.regularizer_wide + tf.nn.l2_loss(w)
               
    
    
    
#       co-occurrence of categorical features
        for i in range(20):
            
            tmpvoca1 = self.N_DISC_VOCA[i] 
            
            for j in range(i+1,20):
                
                tmpvoca2 = self.N_DISC_VOCA[j]
                
                with tf.variable_scope("cooccur"+str(i)+str(j)):
                    
                    
#                   vector approximate of co-occurrence matrix 
                    w1 = tf.Variable(tf.random_normal([self.N_CLASS, tmpvoca1, 1],\
                                            stddev=math.sqrt(2.0/float( tmpvoca1 ))) )
                    
                    w2 = tf.Variable(tf.random_normal([self.N_CLASS, 1, tmpvoca2],\
                                            stddev=math.sqrt(2.0/float( tmpvoca2 ))) )
            
                    
                    b= tf.Variable(tf.zeros( [ self.N_CLASS ] ))

                    
                    cooc_mat = []
                    for k in range(self.N_CLASS):
                        cooc_mat.append( tf.reshape( tf.matmul(w1[k],w2[k]), [-1,1] ) )
                        
#                   n_class by voca1*voca2  
        
                    cooc_vec = tf.stack(cooc_mat)
                    cooc_vec = tf.reshape(cooc_vec, [self.N_CLASS, -1] )
                    cooc_vec = tf.transpose(cooc_vec,[1,0])
                    
#                   build the idx of co-occurrence features
                    cooc_idx = self.dx_trans[i] * tmpvoca2 + self.dx_trans[j]
    
                    if i==0 and j==1:
                        cooc_wsum  = tf.nn.embedding_lookup( cooc_vec, cooc_idx ) + b
#                       L2 
                        self.regularizer_cooc   = tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2) 
                    else:
                        cooc_wsum += tf.nn.embedding_lookup( cooc_vec, cooc_idx ) + b
#                       L2
                        self.regularizer_cooc += (tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2))
    
#       non-linear 
        cooc_wsum = tf.nn.relu( cooc_wsum )
        
        
        
#       embedding of categorical features        
        tmp_embeded = []                
        for i in range(self.N_DISC):
            
            with tf.variable_scope("embedding"+str(i)):
                w= tf.Variable(\
                        tf.random_normal([self.N_DISC_VOCA[i], self.N_EMBED],\
                        stddev=math.sqrt(2.0/float(self.N_DISC_VOCA[i]))) )
#                     tf.random_uniform([self.N_DISC_VOCA, self.N_EMBED],-1.0, 1.0 )
                              
                tmp_embeded.append( tf.nn.embedding_lookup( w, self.dx_trans[i] ) )
                
        dx_embeded = tf.concat(tmp_embeded, 1)
        
        
#       embedding hidden layers
        with tf.variable_scope("embed_h0"):
            w= tf.Variable(tf.random_normal([self.N_TOTAL_EMBED, n_hidden_list[0]],\
                        stddev=math.sqrt(2.0/float(self.N_CONTI)))) 
            b= tf.Variable(tf.zeros( [ n_hidden_list[0] ] ))
            h_embed = tf.nn.relu( tf.add( tf.matmul(dx_embeded, w),b) )
            
#           L2   
            self.regularizer = tf.nn.l2_loss(w)
                
        for i in range(1, self.N_HIDDEN_LAYERS):
            
            with tf.variable_scope("embed_h"+str(i)):
                w= tf.Variable(\
                        tf.random_normal([n_hidden_list[i-1],n_hidden_list[i]],\
                                stddev=math.sqrt(2.0/float(n_hidden_list[i-1])))) 
                b= tf.Variable(tf.zeros( [n_hidden_list[i]] ))
                h_embed = tf.nn.relu( tf.add( tf.matmul(h_embed, w),b) )       
            
#               L2   
                self.regularizer += tf.nn.l2_loss(w)
        
        
        
        
        
#       feature interaction
        for i in range(self.N_DISC):
            
            with tf.variable_scope("interact"+str(i)):
                w= tf.Variable(\
                        tf.random_normal([self.N_DISC_VOCA[i], self.N_CONTI],\
                        stddev=math.sqrt(2.0/float(self.N_DISC_VOCA[i]))) )
#                     tf.random_uniform([self.N_DISC_VOCA, self.N_EMBED],-1.0, 1.0 )
        
                if i==0:
                    inter_wsum  = tf.nn.embedding_lookup( w, self.dx_trans[i] )
#                   L2 
                    self.regularizer_inter  = tf.nn.l2_loss(w)
            
                else:
                    inter_wsum += tf.nn.embedding_lookup( w, self.dx_trans[i] )
#                   L2 
                    self.regularizer_inter += tf.nn.l2_loss(w)

#       add noise bias
        singleCol_h = tf.reduce_sum( self.cx * inter_wsum, 1   )
        tmp_h=[]
        for i in range( n_hidden_list[0] ):
            with tf.variable_scope("noise"+str( i )):
                noise_b = tf.Variable(tf.random_normal([ ],\
                        stddev=math.sqrt(2.0/float(n_hidden_list[0]))))
                tmp_h.append( singleCol_h + noise_b ) 
                
#               L2  
                self.regularizer_inter += tf.nn.l2_loss(noise_b)
        
#         tmp_h=[]
#         for i in range( n_hidden_list[0] ):
#             with tf.variable_scope("noise"+str( i )):
#                 noise_b = tf.Variable(tf.random_normal([ self.N_CONTI ],\
#                         stddev=math.sqrt(2.0/float(n_hidden_list[0]))))
#                 tmp_h.append( tf.reduce_sum( self.cx * (inter_wsum + noise_b), 1 ) ) 
        
        tmp_h = tf.stack( tmp_h )
        tmp_h = tf.transpose( tmp_h, [1,0])
            
        
#       interaction hidden layers


#         singleCol_h = tf.reduce_sum( self.cx * inter_wsum, 1   )
        
#         tmp_h = [ singleCol_h ]*n_hidden_list[0]
        
#         tmp_h = tf.stack( tmp_h )
#         tmp_h = tf.transpose( tmp_h, [1,0])
        
        
        with tf.variable_scope("inter_h0"):
            w = tf.Variable(tf.random_normal([self.N_CONTI, n_hidden_list[0]],\
                        stddev=math.sqrt(2.0/float(self.N_CONTI)))) 
                        
            b = tf.Variable(tf.zeros( [ n_hidden_list[0] ] ))
            
            h_inter = tf.matmul(self.cx, w)
            h_inter = h_inter + tmp_h
            h_inter = tf.nn.relu( h_inter + b)
                             
#           L2  
            self.regularizer += tf.nn.l2_loss(w)
            
        for i in range(1, self.N_HIDDEN_LAYERS):
            
            with tf.variable_scope("inter_h"+str(i)):
                w = tf.Variable(\
                        tf.random_normal([n_hidden_list[i-1],n_hidden_list[i]],\
                                stddev=math.sqrt(2.0/float(n_hidden_list[i-1])))) 
                b = tf.Variable(tf.zeros( [n_hidden_list[i]] ))
                h_inter = tf.nn.relu( tf.add( tf.matmul(h_inter, w),b) )
        
#               L2
                self.regularizer += tf.nn.l2_loss(w)
    
    
#       dropout
#       h = tf.nn.dropout(h, self.keep_prob)
        
        
#       output layer  
        h = tf.concat([h_embed, h_inter], 1)
        
        with tf.variable_scope("output"):
            
            w = tf.Variable(tf.random_normal([n_hidden_list[self.N_HIDDEN_LAYERS-1]*2,\
                                             self.N_CLASS],\
            stddev=math.sqrt(2.0/float(n_hidden_list[self.N_HIDDEN_LAYERS-1])))) 
            b = tf.Variable(tf.zeros( [ self.N_CLASS ] ))
            
            h = tf.matmul(h, w)
#             h = tf.add( h, dx_wsum )
            h = tf.add( h, cooc_wsum )
            h = tf.add( h, b )
        
            self.logit = h
            
#           L2  
            self.regularizer += tf.nn.l2_loss(w) 
    
    
#       overall L2
        self.regularizer += (self.regularizer_cooc + self.regularizer_inter)
#       self.regularizer_wide + 
         
        
#   initialize loss and optimization operations for training
    def train_ini(self):
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(\
                                                    logits=self.logit, labels=self.y) \
                                  + self.L2*self.regularizer )
        self.optimizer = \
        tf.train.AdadeltaOptimizer(learning_rate = self.LEARNING_RATE).minimize(self.cost)
#       tf.train.RMSPropOptimizer(learning_rate = self.LEARNING_RATE).minimize(self.cost)
#         tf.train.AdamOptimizer(learning_rate = self.lr).minimize(self.cost)
#         tf.train.RMSPropOptimizer(learning_rate = self.lr).minimize(self.cost)
#         tf.train.AdadeltaOptimizer(learning_rate = self.lr).minimize(self.cost)
#         tf.train.AdamOptimizer(learning_rate = self.lr).minimize(self.cost)
#         tf.train.GradientDescentOptimizer(learning_rate = self.lr).minimize(self.cost)
        
        self.init = tf.global_variables_initializer()
        sess.run( self.init )
        
#   training on batch of data
    def train_batch(self, dx_batch,cx_batch, y_batch, keep_prob, lr):
        
        _,c = sess.run([self.optimizer,self.cost],\
                       feed_dict={self.dx:dx_batch, \
                                  self.cx:cx_batch, self.y:y_batch,\
                                  self.lr:lr,\
                                  self.keep_prob:keep_prob\
                                 })
        
        return c
        
#   initialize inference         
    def inference_ini(self):
        self.correct_prediction = tf.equal(tf.argmax(self.logit,1), tf.argmax(self.y,1))
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, "float"))
        
#   infer givn testing data    
    def inference(self, dx_test, cx_test, y_test, keep_prob):
        return sess.run([self.accuracy], feed_dict={self.dx:dx_test,\
                                                  self.cx:cx_test, self.y:y_test,\
                                                  self.keep_prob:keep_prob\
                                                   })
        
#   unit_test
    def test(self, dx_test, cx_test, y_test ):
        self.init = tf.global_variables_initializer()
        sess.run( self.init )
        return sess.run( [self.tmp1], \
                 feed_dict={self.dx:dx_test, self.cx:cx_test, self.y:y_test})
        
        

In [4]:
# wide, embedding, co-occurrence and feature interaction 
# version 2: individual hidden layers on embedding and interaction

para_n_epoch = 1200

# tunable parameters

#   representation ability
para_n_hidden_list = [ 128,64,32,16 ]
#     128,16,8 ]
para_lr = 0.075
para_n_embedding = 2
#   regularization
para_batch_size = 32
para_keep_prob = 1.0
para_l2 = 0.25

# fixed parameters
para_n_class = 8
para_n_disc = 80
para_n_conti = 106
para_n_disc_voca = [8]*para_n_disc

# evaluation parameters
para_eval_byepoch = 10


with tf.Session() as sess:
    
    clf = InterNN_IndiH( para_n_conti, para_n_disc, para_n_class, \
                      para_n_disc_voca, sess, para_n_embedding, para_n_hidden_list,\
                        para_lr, para_l2, para_batch_size)
    clf.train_ini()
    clf.inference_ini()
    
    para_cur_lr=para_lr
    
    total_cnt= np.shape(cxtrain)[0]
    total_batch = int(total_cnt/para_batch_size)
    
    total_idx=range(total_cnt)
    
    
#   training cycle
    for epoch in range(para_n_epoch):
        
        tmpc=0.0
        
#       shuffle traning instances each epoch  
        np.random.shuffle(total_idx)
        
#       Loop over all batches
        for i in range(total_batch):
            
            batch_idx = total_idx[ i*para_batch_size: (i+1)*para_batch_size ] 
            
            batch_cx = cxtrain[ batch_idx ]
            batch_dx = dxtrain[ batch_idx ]
            batch_y  =  ytrain[ batch_idx ]
            
            tmpc += clf.train_batch( batch_dx, batch_cx, batch_y, para_keep_prob,\
                                     para_cur_lr )
        
        if epoch%para_eval_byepoch != 0:
            continue
        
        tmp_test_acc  = clf.inference(dxtest, cxtest, ytest, para_keep_prob) 
        tmp_train_acc = clf.inference(dxtrain, cxtrain, ytrain, para_keep_prob) 
        
        print "loss on epoch ", epoch, " : ", 1.0*tmpc/total_batch, tmp_test_acc,\
        tmp_train_acc
        
    print "Optimization Finished!"

In [ ]:
[128, 64, 32, 16]

0.075 0.25

loss on epoch  0  :  2980.92136863 [0.1919585] [0.1814644]
loss on epoch  10  :  1920.12954881 [0.230869] [0.22614586]
loss on epoch  20  :  999.76868467 [0.21271077] [0.22946094]
loss on epoch  30  :  444.889363607 [0.26848248] [0.25036034]
loss on epoch  40  :  171.663069054 [0.2542153] [0.27601615]
loss on epoch  50  :  60.137812067 [0.26718548] [0.27169213]
loss on epoch  60  :  22.0798286862 [0.2775616] [0.26780051]
loss on epoch  70  :  10.2165232632 [0.29053178] [0.26996252]
loss on epoch  80  :  5.84338207598 [0.30090791] [0.2711156]
loss on epoch  90  :  3.5995438695 [0.31128404] [0.27644855]
loss on epoch  100  :  2.41466889116 [0.31128404] [0.27875468]
loss on epoch  110  :  2.02685040567 [0.31258106] [0.28279042]
loss on epoch  120  :  1.87719280852 [0.30220494] [0.27961949]
loss on epoch  130  :  1.81999501917 [0.30350193] [0.27875468]
loss on epoch  140  :  1.79834823973 [0.26459143] [0.26160276]
loss on epoch  150  :  1.78521902142 [0.26329443] [0.26549438]
loss on epoch  160  :  1.77017820268 [0.27107653] [0.26145864]
loss on epoch  170  :  1.75266613894 [0.25551233] [0.24589218]
loss on epoch  180  :  1.74597876878 [0.25551233] [0.24589218]
loss on epoch  190  :  1.74167404517 [0.25551233] [0.24589218]
loss on epoch  200  :  1.73957077607 [0.25551233] [0.24589218]
loss on epoch  210  :  1.7380380349 [0.25551233] [0.24589218]
loss on epoch  220  :  1.73560580611 [0.25551233] [0.24589218]
loss on epoch  230  :  1.73450110853 [0.25551233] [0.24589218]
loss on epoch  240  :  1.73394604634 [0.25551233] [0.24589218]



0.04 0.15
loss on epoch  0  :  1813.85026268 [0.14007781] [0.17223984]
loss on epoch  10  :  1444.94381431 [0.21919584] [0.2164889]
loss on epoch  20  :  1027.52567743 [0.22438392] [0.22960508]
loss on epoch  30  :  674.013945685 [0.23346303] [0.24055924]
loss on epoch  40  :  413.576934814 [0.28145266] [0.25569329]
loss on epoch  50  :  238.739363847 [0.28793773] [0.25381956]
loss on epoch  60  :  131.519887253 [0.29312581] [0.26707986]
loss on epoch  70  :  70.8839188328 [0.28404668] [0.27601615]
loss on epoch  80  :  38.5956713888 [0.28664073] [0.27500722]
loss on epoch  90  :  22.4106277007 [0.28015563] [0.27659267]
loss on epoch  100  :  14.5088500005 [0.28274968] [0.28581724]
loss on epoch  110  :  10.4810321773 [0.28145266] [0.28322282]
loss on epoch  120  :  8.26340355697 [0.30479896] [0.28898817]
loss on epoch  130  :  6.84820767464 [0.30479896] [0.29677141]
loss on epoch  140  :  5.76905129132 [0.31776914] [0.29172671]
loss on epoch  150  :  4.88457653699 [0.30739298] [0.30008647]
loss on epoch  160  :  4.13123914158 [0.30998704] [0.29994234]
loss on epoch  170  :  3.48497954452 [0.31776914] [0.30628422]
loss on epoch  180  :  2.94781986431 [0.31128404] [0.31507638]
loss on epoch  190  :  2.55362282528 [0.30479896] [0.3202652]
loss on epoch  200  :  2.3066153736 [0.30350193] [0.32559815]
loss on epoch  210  :  2.14339051313 [0.29701686] [0.33294898]
loss on epoch  220  :  2.03205191151 [0.30479896] [0.34202939]
loss on epoch  230  :  1.95423617628 [0.29571983] [0.35773998]
loss on epoch  240  :  1.88982394338 [0.29312581] [0.37835112]
loss on epoch  250  :  1.83355036268 [0.28793773] [0.39939463]
loss on epoch  260  :  1.78425233232 [0.27367055] [0.42029405]
loss on epoch  270  :  1.74030464281 [0.27237353] [0.43787834]
loss on epoch  280  :  1.70205291443 [0.27107653] [0.47492072]
loss on epoch  290  :  1.66948302587 [0.27237353] [0.4876045]
loss on epoch  300  :  1.64072179629 [0.26848248] [0.53617758]

0.04 0.25

loss on epoch  0  :  3022.73503847 [0.13748379] [0.11617181]
loss on epoch  10  :  2354.38716295 [0.22438392] [0.21951571]
loss on epoch  20  :  1630.79777922 [0.25291827] [0.24113578]
loss on epoch  30  :  1041.21271317 [0.2775616] [0.2706832]
loss on epoch  40  :  619.60531588 [0.28534371] [0.26909772]
loss on epoch  50  :  344.483751791 [0.27237353] [0.26823291]
loss on epoch  60  :  179.307656889 [0.27626458] [0.26044971]
loss on epoch  70  :  87.79225028 [0.27885863] [0.26102623]
loss on epoch  80  :  40.9197322174 [0.2775616] [0.25771114]
loss on epoch  90  :  18.6923910865 [0.25810635] [0.248919]
loss on epoch  100  :  8.86420894111 [0.25551233] [0.24560392]
loss on epoch  110  :  4.75540789851 [0.25940338] [0.24949554]
loss on epoch  120  :  3.10697445273 [0.30609599] [0.2822139]
loss on epoch  130  :  2.41909366625 [0.31517509] [0.28596136]
loss on epoch  140  :  2.10791990382 [0.31258106] [0.28985298]
loss on epoch  150  :  1.97171778977 [0.32555124] [0.29230326]
loss on epoch  160  :  1.913080184 [0.32295719] [0.29518592]
loss on epoch  170  :  1.87703902743 [0.32036316] [0.29720381]
loss on epoch  180  :  1.85110016498 [0.32166019] [0.29605073]
loss on epoch  190  :  1.83072735976 [0.32295719] [0.29806861]
loss on epoch  200  :  1.81598060661 [0.32425421] [0.29806861]
loss on epoch  210  :  1.80346884716 [0.32684824] [0.29691553]
loss on epoch  220  :  1.79394866085 [0.32684824] [0.29648313]
loss on epoch  230  :  1.78761300058 [0.32295719] [0.29734793]
loss on epoch  240  :  1.78324816238 [0.32684824] [0.29720381]
loss on epoch  250  :  1.77923020058 [0.32555124] [0.2959066]
loss on epoch  260  :  1.77586780433 [0.32425421] [0.2950418]
loss on epoch  270  :  1.77452967288 [0.32166019] [0.2950418]
loss on epoch  280  :  1.77241332387 [0.32555124] [0.294177]
loss on epoch  290  :  1.77146135657 [0.32036316] [0.29345632]
loss on epoch  300  :  1.77025835216 [0.32684824] [0.29287979]
loss on epoch  310  :  1.7693916653 [0.32166019] [0.29143846]
loss on epoch  320  :  1.76785501617 [0.32166019] [0.29287979]
loss on epoch  330  :  1.76768690734 [0.32425421] [0.29187086]
loss on epoch  340  :  1.76657673054 [0.32166019] [0.29287979]
loss on epoch  350  :  1.76623662037 [0.31517509] [0.29360047]
loss on epoch  360  :  1.76513794175 [0.31517509] [0.29273567]
loss on epoch  370  :  1.76418453345 [0.31776914] [0.29316807]
loss on epoch  380  :  1.76357118675 [0.31647211] [0.29215911]
loss on epoch  390  :  1.76390206262 [0.31906614] [0.29244739]
loss on epoch  400  :  1.76301965007 [0.31776914] [0.29273567]

0.065 0.25
loss on epoch  0  :  3000.83908759 [0.17120622] [0.17598732]
loss on epoch  10  :  2021.8409729 [0.26070037] [0.24084751]
loss on epoch  20  :  1133.03225482 [0.28274968] [0.27241281]
loss on epoch  30  :  553.712874236 [0.28015563] [0.28077257]
loss on epoch  40  :  238.452506313 [0.28664073] [0.28250217]
loss on epoch  50  :  92.1786404009 [0.29701686] [0.28797925]
loss on epoch  60  :  34.2706054582 [0.29961088] [0.29244739]
loss on epoch  70  :  14.0935545409 [0.30350193] [0.29129431]
loss on epoch  80  :  7.20146068158 [0.30479896] [0.29720381]
loss on epoch  90  :  4.41210737714 [0.30479896] [0.31651774]
loss on epoch  100  :  3.03185175194 [0.31906614] [0.32862496]
loss on epoch  110  :  2.36303060474 [0.31906614] [0.32271549]
loss on epoch  120  :  2.07406861087 [0.32295719] [0.32876909]
loss on epoch  130  :  1.94719672369 [0.31517509] [0.32848084]
loss on epoch  140  :  1.88129817281 [0.32295719] [0.32675123]
loss on epoch  150  :  1.84270298481 [0.31647211] [0.32689536]
loss on epoch  160  :  1.81902885437 [0.31258106] [0.32545403]
loss on epoch  170  :  1.80198419536 [0.31128404] [0.32430094]
loss on epoch  180  :  1.79108801539 [0.30998704] [0.3240127]
loss on epoch  190  :  1.78217020355 [0.31258106] [0.32271549]
loss on epoch  200  :  1.7755769138 [0.31258106] [0.32098588]
loss on epoch  210  :  1.77042962887 [0.30739298] [0.32170653]
loss on epoch  220  :  1.76636268806 [0.31647211] [0.3198328]
loss on epoch  230  :  1.7633897894 [0.31647211] [0.32185069]
loss on epoch  240  :  1.76053985401 [0.31647211] [0.32242721]
loss on epoch  250  :  1.75875376993 [0.31776914] [0.32228309]
loss on epoch  260  :  1.75633004749 [0.32036316] [0.32170653]
loss on epoch  270  :  1.75460924263 [0.31647211] [0.32285962]
loss on epoch  280  :  1.75377040605 [0.31387809] [0.32213894]
loss on epoch  290  :  1.75220181986 [0.32036316] [0.3240127]
loss on epoch  300  :  1.75125846708 [0.31647211] [0.32257134]
loss on epoch  310  :  1.75043590312 [0.31776914] [0.32300374]
loss on epoch  320  :  1.74990666244 [0.31258106] [0.32314789]
loss on epoch  330  :  1.74844655615 [0.31517509] [0.3253099]
loss on epoch  340  :  1.74757094129 [0.31387809] [0.32430094]
loss on epoch  350  :  1.74773016682 [0.30739298] [0.32300374]
loss on epoch  360  :  1.74644510117 [0.31128404] [0.32675123]
loss on epoch  370  :  1.74650620586 [0.31258106] [0.32257134]
loss on epoch  380  :  1.74503148982 [0.30869001] [0.32473335]
loss on epoch  390  :  1.74497717398 [0.31258106] [0.32458922]
loss on epoch  400  :  1.74421742779 [0.30090791] [0.32776016]
loss on epoch  410  :  1.74366191581 [0.30479896] [0.32646295]
loss on epoch  420  :  1.74315313774 [0.31128404] [0.32617468]
loss on epoch 

0.065 0.2

loss on epoch  0  :  2415.1183709 [0.1841764] [0.19054483]
loss on epoch  10  :  1640.92718393 [0.25032425] [0.23508215]
loss on epoch  20  :  923.604782952 [0.22697794] [0.23277602]
loss on epoch  30  :  452.972128691 [0.22308689] [0.25439608]
loss on epoch  40  :  196.297149658 [0.24902724] [0.253243]
loss on epoch  50  :  76.8364204124 [0.2697795] [0.26621506]
loss on epoch  60  :  29.1407067687 [0.26588845] [0.28682616]
loss on epoch  70  :  12.2159188324 [0.30350193] [0.30974343]
loss on epoch  80  :  6.27734933297 [0.31647211] [0.32257134]
loss on epoch  90  :  3.86496341008 [0.32295719] [0.31767079]
loss on epoch  100  :  2.80003212668 [0.31128404] [0.32516575]
loss on epoch  110  :  2.27337547695 [0.31647211] [0.3304987]
loss on epoch  120  :  2.02081030331 [0.31517509] [0.33323723]
loss on epoch  130  :  1.91915968833 [0.31776914] [0.33712885]
loss on epoch  140  :  1.86473320866 [0.31647211] [0.33943498]
loss on epoch  150  :  1.83126418017 [0.31906614] [0.34952435]
loss on epoch  160  :  1.80747572139 [0.32295719] [0.34938022]
loss on epoch  170  :  1.78739842331 [0.31776914] [0.35356009]
loss on epoch  180  :  1.7721317703 [0.30739298] [0.36552322]
loss on epoch  190  :  1.75847657356 [0.31647211] [0.37820697]
loss on epoch  200  :  1.7453189426 [0.30869001] [0.39607957]
loss on epoch  210  :  1.73079885542 [0.31128404] [0.42490631]
loss on epoch  220  :  1.71684144493 [0.31387809] [0.4517152]
loss on epoch  230  :  1.6998641215 [0.30090791] [0.47982126]
loss on epoch  240  :  1.68529631142 [0.29701686] [0.51052177]
loss on epoch  250  :  1.66939626193 [0.28664073] [0.54151052]
loss on epoch  260  :  1.65720480791 [0.28015563] [0.56658977]
loss on epoch 

0.055 0.2
loss on epoch  0  :  2405.1642829 [0.15434501] [0.16402422]
loss on epoch  10  :  1822.05523569 [0.25680932] [0.23551455]
loss on epoch  20  :  1206.30413084 [0.25032425] [0.26044971]
loss on epoch  30  :  727.089557224 [0.2619974] [0.24704526]
loss on epoch  40  :  403.501825827 [0.24124514] [0.23205535]
loss on epoch  50  :  206.482957063 [0.27107653] [0.23998271]
loss on epoch  60  :  97.5921313674 [0.27885863] [0.24156818]
loss on epoch  70  :  42.7869045116 [0.26848248] [0.25108099]
loss on epoch  80  :  17.7319767475 [0.26848248] [0.27284521]
loss on epoch  90  :  7.41176229936 [0.28015563] [0.28682616]
loss on epoch  100  :  3.63745385519 [0.30479896] [0.30023062]
loss on epoch  110  :  2.40975147375 [0.31128404] [0.31046411]
loss on epoch  120  :  2.02496744968 [0.31128404] [0.31925628]
loss on epoch  130  :  1.89303689698 [0.31906614] [0.32300374]
loss on epoch  140  :  1.83951033983 [0.31128404] [0.3253099]
loss on epoch  150  :  1.8114690295 [0.31906614] [0.33107525]
loss on epoch  160  :  1.79382089277 [0.30998704] [0.3326607]
loss on epoch  170  :  1.78107121476 [0.30739298] [0.34145287]
loss on epoch  180  :  1.7699627512 [0.31387809] [0.34275007]
loss on epoch  190  :  1.76237651982 [0.31647211] [0.34260595]
loss on epoch  200  :  1.75571231323 [0.32036316] [0.343759]
loss on epoch  210  :  1.74952059598 [0.31647211] [0.35082155]
loss on epoch  220  :  1.7445356316 [0.32295719] [0.35240704]

0.055 0.1
loss on epoch  0  :  1214.3227816 [0.18158236] [0.19731911]
loss on epoch  10  :  883.076656483 [0.24124514] [0.24819833]
loss on epoch  20  :  547.064853527 [0.2464332] [0.25540501]
loss on epoch  30  :  300.342731476 [0.2464332] [0.26722398]
loss on epoch  40  :  147.775008308 [0.25551233] [0.27385414]
loss on epoch  50  :  66.0783908632 [0.26329443] [0.27745748]
loss on epoch  60  :  27.6277261134 [0.26459143] [0.27486306]
loss on epoch  70  :  11.4821305098 [0.29053178] [0.27875468]
loss on epoch  80  :  5.25120859897 [0.30090791] [0.28913233]
loss on epoch  90  :  2.95559102407 [0.30869001] [0.31118479]
loss on epoch  100  :  2.20472129738 [0.31776914] [0.32920149]
loss on epoch  110  :  1.96927175202 [0.30479896] [0.33900261]
loss on epoch  120  :  1.88147748841 [0.31128404] [0.3483713]
loss on epoch  130  :  1.83761433043 [0.30998704] [0.35701931]
loss on epoch  140  :  1.80635545817 [0.30739298] [0.37042376]
loss on epoch  150  :  1.77693002367 [0.30869001] [0.39391756]
loss on epoch  160  :  1.74524878186 [0.29701686] [0.41784376]
loss on epoch  170  :  1.70953490557 [0.28274968] [0.45358893]
loss on epoch  180  :  1.67064969518 [0.28404668] [0.49899107]
loss on epoch  190  :  1.63381898017 [0.29961088] [0.54597867]
loss on epoch  200  :  1.601610325 [0.27107653] [0.58633614]
loss on epoch 


0.055 0.3
loss on epoch  0  :  3615.92828143 [0.20622568] [0.1925627]
loss on epoch  10  :  2570.61795496 [0.25032425] [0.24517152]
loss on epoch  20  :  1554.36551073 [0.23216602] [0.23263188]
loss on epoch  30  :  836.849906356 [0.22568093] [0.23767656]
loss on epoch  40  :  403.61595832 [0.29571983] [0.2528106]
loss on epoch  50  :  173.99371896 [0.27626458] [0.24920726]
loss on epoch  60  :  66.8145278295 [0.29961088] [0.25583741]
loss on epoch  70  :  23.1336858714 [0.29701686] [0.26160276]
loss on epoch  80  :  7.77529838792 [0.29571983] [0.26751226]
loss on epoch  90  :  3.21875131572 [0.25551233] [0.24589218]
loss on epoch  100  :  2.10442849663 [0.25551233] [0.24589218]
loss on epoch  110  :  1.87542454788 [0.25551233] [0.24589218]
loss on epoch  120  :  1.82239278047 [0.25551233] [0.24589218]
loss on epoch  130  :  1.78458884524 [0.25551233] [0.24589218]
loss on epoch  140  :  1.77022338686 [0.25551233] [0.24589218]
loss on epoch  150  :  1.7620690901 [0.25551233] [0.24589218]
loss on epoch  160  :  1.75576829027 [0.25551233] [0.24589218]
loss on epoch  170  :  1.75212164278 [0.25551233] [0.24589218]
loss on epoch  180  :  1.74801673105 [0.25551233] [0.24589218]
loss on epoch  190  :  1.74692355317 [0.25551233] [0.24589218]
loss on epoch  200  :  1.74399915834 [0.25551233] [0.24589218]
loss on epoch  210  :  1.74284921587 [0.25551233] [0.24589218]
loss on epoch  220  :  1.7411341038 [0.25551233] [0.24589218]
loss on epoch  230  :  1.73962150956 [0.25551233] [0.24589218]
loss on epoch  240  :  1.73871049395 [0.25551233] [0.24589218]
loss on epoch  250  :  1.73769390528 [0.25551233] [0.24589218]
loss on epoch  260  :  1.73628741006 [0.25551233] [0.24589218]
loss on epoch  270  :  1.7356295685 [0.25551233] [0.24589218]


[ 64,32,16 ]

0.055 0.3

loss on epoch  0  :  3538.809611 [0.22438392] [0.21504757]
loss on epoch  10  :  2595.15303209 [0.19844358] [0.19674258]
loss on epoch  20  :  1622.10589882 [0.26070037] [0.25454021]
loss on epoch  30  :  898.301842018 [0.26588845] [0.26549438]
loss on epoch  40  :  443.957723688 [0.2697795] [0.26996252]
loss on epoch  50  :  196.289967007 [0.2697795] [0.27443066]
loss on epoch  60  :  78.9105699327 [0.27496758] [0.28062841]
loss on epoch  70  :  30.2949818505 [0.28664073] [0.29057366]
loss on epoch  80  :  12.5827726744 [0.30998704] [0.29835686]
loss on epoch  90  :  6.71447044611 [0.30998704] [0.30080715]
loss on epoch  100  :  4.66515111482 [0.32166019] [0.29806861]
loss on epoch  110  :  3.67433682526 [0.32166019] [0.29605073]
loss on epoch  120  :  2.96085031938 [0.30998704] [0.30268088]
loss on epoch  130  :  2.41947878842 [0.32036316] [0.31392333]
loss on epoch  140  :  2.07686446276 [0.32295719] [0.32040933]
loss on epoch  150  :  1.90246647044 [0.32555124] [0.32545403]
loss on epoch  160  :  1.82202365553 [0.32036316] [0.3240127]
loss on epoch  170  :  1.78665670753 [0.32036316] [0.32891324]
loss on epoch  180  :  1.76989234432 [0.32555124] [0.32761604]
loss on epoch  190  :  1.76004665704 [0.31647211] [0.32948977]
loss on epoch  200  :  1.75389075997 [0.32425421] [0.32963389]
loss on epoch  210  :  1.74937579588 [0.32555124] [0.32876909]
loss on epoch  220  :  1.74488390209 [0.31906614] [0.33121938]
loss on epoch  230  :  1.74217996388 [0.32166019] [0.3304987]
loss on epoch  240  :  1.73865111503 [0.31517509] [0.3304987]
loss on epoch  250  :  1.73666188066 [0.31776914] [0.33107525]
loss on epoch  260  :  1.7345862074 [0.32555124] [0.33208418]
loss on epoch  270  :  1.73225748318 [0.32425421] [0.33294898]
loss on epoch  280  :  1.72988482482 [0.32555124] [0.33323723]
loss on epoch  290  :  1.72851397428 [0.31906614] [0.3368406]
loss on epoch  300  :  1.72767322538 [0.31517509] [0.33885846]
loss on epoch  310  :  1.72534208883 [0.31776914] [0.34029979]

0.045 0.3

loss on epoch  0  :  3527.2062457 [0.16342412] [0.17425771]
loss on epoch  10  :  2692.3897106 [0.21530479] [0.23983857]
loss on epoch  20  :  1796.63398347 [0.25940338] [0.24070337]
loss on epoch  30  :  1089.10582818 [0.25940338] [0.25929663]
loss on epoch  40  :  606.953802321 [0.26329443] [0.26722398]
loss on epoch  50  :  311.657185166 [0.27496758] [0.28365523]
loss on epoch  60  :  147.537786978 [0.25810635] [0.26376477]
loss on epoch  70  :  64.825095318 [0.27107653] [0.26088211]
loss on epoch  80  :  26.9724987613 [0.26848248] [0.25987315]
loss on epoch  90  :  11.2345146471 [0.25291827] [0.24791007]
loss on epoch  100  :  5.22163809891 [0.25162128] [0.24776593]
loss on epoch  110  :  3.08083092504 [0.25810635] [0.25295475]
loss on epoch  120  :  2.34396255127 [0.26329443] [0.2541078]
loss on epoch  130  :  2.06469206622 [0.30998704] [0.28019601]
loss on epoch  140  :  1.93668902803 [0.31517509] [0.28423178]
loss on epoch  150  :  1.8661926019 [0.30998704] [0.28552896]
loss on epoch  160  :  1.82431523943 [0.31387809] [0.28624964]
loss on epoch  170  :  1.79818637559 [0.31647211] [0.28452003]
loss on epoch  180  :  1.78158993081 [0.30998704] [0.28639376]
loss on epoch  190  :  1.77237643522 [0.31128404] [0.28783512]
loss on epoch  200  :  1.76762638821 [0.31517509] [0.28697032]
loss on epoch  210  :  1.76267459106 [0.31647211] [0.28956473]
loss on epoch  220  :  1.76056111521 [0.31776914] [0.29129431]
loss on epoch  230  :  1.75803216022 [0.31906614] [0.29518592]
loss on epoch  240  :  1.75444278287 [0.31776914] [0.29605073]
loss on epoch  250  :  1.751945553 [0.31906614] [0.29619488]
loss on epoch  260  :  1.75040145715 [0.32036316] [0.29749209]
loss on epoch  270  :  1.74882869698 [0.31906614] [0.30123955]
loss on epoch  280  :  1.74721703154 [0.31906614] [0.30282503]
loss on epoch  290  :  1.74496475635 [0.32036316] [0.30599597]
loss on epoch  300  :  1.74315591046 [0.32295719] [0.3078697]
loss on epoch  310  :  1.74128439967 [0.32814527] [0.30988759]
loss on epoch  320  :  1.73928736316 [0.32295719] [0.31046411]
loss on epoch  330  :  1.73696815085 [0.32555124] [0.31291437]
loss on epoch  340  :  1.73604475679 [0.32425421] [0.31622946]
loss on epoch  350  :  1.73450473117 [0.32425421] [0.31752667]
loss on epoch  360  :  1.73281933864 [0.32684824] [0.31824735]
loss on epoch  370  :  1.73161647386 [0.32814527] [0.3202652]
loss on epoch  380  :  1.73021129215 [0.32684824] [0.32199481]
loss on epoch  390  :  1.72862869887 [0.32555124] [0.32329202]
loss on epoch  400  :  1.72775928069 [0.32425421] [0.32502162]

0.065 0.25

loss on epoch  0  :  2921.43431487 [0.15434501] [0.1729605]
loss on epoch  10  :  1991.26863494 [0.21660182] [0.23277602]
loss on epoch  20  :  1114.40263423 [0.25940338] [0.25165755]
loss on epoch  30  :  537.532615096 [0.23216602] [0.24963966]
loss on epoch  40  :  225.796810857 [0.26588845] [0.25454021]
loss on epoch  50  :  82.8712526604 [0.25680932] [0.24675699]
loss on epoch  60  :  27.1193968367 [0.24513619] [0.24401845]
loss on epoch  70  :  8.71390389955 [0.23605707] [0.2421447]
loss on epoch  80  :  3.54491505689 [0.23994812] [0.24099164]
loss on epoch  90  :  2.26177679168 [0.23605707] [0.23897377]
loss on epoch  100  :  1.94379551654 [0.23605707] [0.2373883]
loss on epoch  110  :  1.84164449352 [0.24254215] [0.24084751]
loss on epoch  120  :  1.80054399206 [0.30220494] [0.27039492]
loss on epoch  130  :  1.78005009448 [0.31258106] [0.28019601]
loss on epoch  140  :  1.76697096725 [0.31517509] [0.28322282]
loss on epoch  150  :  1.75738607402 [0.31128404] [0.29360047]
loss on epoch  160  :  1.7507715297 [0.30998704] [0.30152783]
loss on epoch  170  :  1.74384902694 [0.31387809] [0.30887863]
loss on epoch  180  :  1.73778158537 [0.30739298] [0.31406745]
loss on epoch  190  :  1.73310478749 [0.31647211] [0.32372442]
loss on epoch  200  :  1.72765880326 [0.32425421] [0.32992217]
loss on epoch  210  :  1.72318710663 [0.32425421] [0.33338138]
loss on epoch  220  :  1.71833285137 [0.32555124] [0.337273]
loss on epoch  230  :  1.71249576189 [0.32295719] [0.34347075]
loss on epoch  240  :  1.70597554854 [0.31906614] [0.35370424]
loss on epoch  250  :  1.69754237157 [0.31776914] [0.36292881]
loss on epoch  260  :  1.68777828581 [0.32166019] [0.37748632]
loss on epoch  270  :  1.67723594882 [0.31258106] [0.38988182]
loss on epoch  280  :  1.66504802969 [0.29312581] [0.41395214]
loss on epoch  290  :  1.65263791548 [0.29442284] [0.42706832]
loss on epoch  300  :  1.64188275017 [0.29053178] [0.44854426]
loss on epoch  310  :  1.63121555635 [0.28404668] [0.47492072]
loss on epoch  320  :  1.61832285479 [0.28664073] [0.48501009]
loss on epoch  330  :  1.61033396864 [0.29961088] [0.5197463]
loss on epoch  340  :  1.60363788296 [0.27885863] [0.52536756]
loss on epoch  350  :  1.59526930584 [0.27885863] [0.54050159]
loss on epoch  360  :  1.59068557951 [0.28145266] [0.58013839]
loss on epoch  370  :  1.5861019448 [0.28015563] [0.56586915]
loss on epoch  380  :  1.58222953644 [0.27626458] [0.5841741]
loss on epoch  390  :  1.57821790819 [0.28534371] [0.58820987]
loss on epoch  400  :  1.57750490087 [0.2775616] [0.60910928]
loss on epoch  410  :  1.57443232724 [0.28793773] [0.61184782]
loss on epoch  420  :  1.57079277381 [0.28923476] [0.61083883]
loss on epoch  430  :  1.56823886048 [0.2775616] [0.62337852]
loss on epoch  440  :  1.56650043858 [0.29312581] [0.6264053]
loss on epoch  450  :  1.56567399369 [0.29053178] [0.62784666]
loss on epoch  460  :  1.56381078285 [0.28923476] [0.62438744]
loss on epoch  470  :  1.56204192672 [0.28274968] [0.62899971]
loss on epoch  480  :  1.56032571473 [0.2775616] [0.63274717]
loss on epoch  490  :  1.56006186593 [0.29053178] [0.63087344]
loss on epoch  500  :  1.55774901145 [0.29182878] [0.62799078]

    
0.045 0.15    

loss on epoch  0  :  1767.57469573 [0.16212711] [0.16488902]
loss on epoch  10  :  1372.31559471 [0.23735408] [0.23637936]
loss on epoch  20  :  931.9710764 [0.25291827] [0.24084751]
loss on epoch  30  :  572.029821325 [0.24773023] [0.24315365]
loss on epoch  40  :  321.629333072 [0.26718548] [0.24618046]
loss on epoch  50  :  166.821157526 [0.27367055] [0.26030555]
loss on epoch  60  :  80.7114824366 [0.26329443] [0.26693571]
loss on epoch  70  :  37.1020909062 [0.26329443] [0.26145864]
loss on epoch  80  :  16.9038463434 [0.27626458] [0.2647737]
loss on epoch  90  :  8.34884393657 [0.30220494] [0.28956473]
loss on epoch  100  :  4.8983892953 [0.30220494] [0.29475352]
loss on epoch  110  :  3.41299184936 [0.29961088] [0.30484289]
loss on epoch  120  :  2.72366370095 [0.30869001] [0.31003171]
loss on epoch  130  :  2.35682953287 [0.30998704] [0.31449986]
loss on epoch  140  :  2.14423345195 [0.32166019] [0.32415682]
loss on epoch  150  :  2.00732905832 [0.31517509] [0.3326607]
loss on epoch  160  :  1.91791861973 [0.30609599] [0.337273]
loss on epoch  170  :  1.856142591 [0.30350193] [0.3488037]
loss on epoch  180  :  1.81220842677 [0.30609599] [0.35932547]
loss on epoch  190  :  1.77689304341 [0.30998704] [0.37287402]
loss on epoch  200  :  1.74662932367 [0.31258106] [0.38930526]
loss on epoch  210  :  1.71966821673 [0.30220494] [0.41106948]
loss on epoch  220  :  1.69432809949 [0.30739298] [0.43528393]
loss on epoch  230  :  1.66779694679 [0.28793773] [0.4586336]
loss on epoch  240  :  1.64370422893 [0.28923476] [0.49149612]
loss on epoch  250  :  1.61674487922 [0.28145266] [0.52176422]
loss on epoch  260  :  1.5938981192 [0.29053178] [0.55232054]
loss on epoch  270  :  1.57478572152 [0.28923476] [0.57336408]
loss on epoch  280  :  1.55752837492 [0.28274968] [0.58316517]
    
[ 128,64,16 ]
0.065 0.25
loss on epoch  0  :  2977.87608959 [0.1919585] [0.15854713]
loss on epoch  10  :  2019.57307943 [0.23605707] [0.23854136]
loss on epoch  20  :  1134.16836096 [0.23735408] [0.25612569]
loss on epoch  30  :  555.670838815 [0.23994812] [0.26059383]
loss on epoch  40  :  240.495475345 [0.25680932] [0.26621506]
loss on epoch  50  :  94.1048941082 [0.26459143] [0.27226865]
loss on epoch  60  :  35.755759822 [0.2697795] [0.28826752]
loss on epoch  70  :  14.8548342254 [0.26848248] [0.29734793]
loss on epoch  80  :  7.34670079196 [0.29571983] [0.30210435]
loss on epoch  90  :  4.27060962386 [0.26718548] [0.29994234]
loss on epoch  100  :  2.92792832631 [0.25680932] [0.28106081]
loss on epoch  110  :  2.28485731284 [0.24254215] [0.25886422]
loss on epoch  120  :  2.00470601519 [0.24124514] [0.2493514]
loss on epoch  130  :  1.89255996931 [0.23346303] [0.24517152]
loss on epoch  140  :  1.83380711134 [0.27885863] [0.26506197]
loss on epoch  150  :  1.78784273896 [0.28923476] [0.29432112]
loss on epoch  160  :  1.74635697349 [0.29442284] [0.32790428]
loss on epoch  170  :  1.70921373754 [0.29053178] [0.37100029]
loss on epoch  180  :  1.67862620012 [0.28664073] [0.39881811]
loss on epoch  190  :  1.6560735686 [0.29053178] [0.41856444]
loss on epoch  200  :  1.63647926019 [0.28923476] [0.43052754]
loss on epoch  210  :  1.62218755097 [0.29701686] [0.44292304]
loss on epoch  220  :  1.60957927947 [0.30998704] [0.44984144]
loss on epoch  230  :  1.59824935319 [0.30090791] [0.45690399]
loss on epoch  240  :  1.58998712345 [0.29442284] [0.46612856]
loss on epoch  250  :  1.58220509743 [0.29182878] [0.47290286]
loss on epoch  260  :  1.57375625273 [0.28274968] [0.47636205]

0.045 0.15

In [3]:
# wide, embedding, co-occurrence and feature interaction
# version 3: embedding + interaction

class InterNN_fuse():
    
    def __init__(self, n_conti, n_disc, n_class,\
                 n_disc_voca, session,n_embedding, n_hidden_list, lr, l2, batch_size):
        
#       build the network graph 
        self.LEARNING_RATE = lr
                
        self.N_CLASS = n_class
        self.N_CONTI = n_conti
        self.N_DISC  = n_disc
        self.N_DISC_VOCA = n_disc_voca
        self.N_EMBED = n_embedding
        self.N_BATCH = batch_size
        self.L2 = l2
        
        self.N_HIDDEN_LAYERS = len(n_hidden_list)
        self.N_TOTAL = self.N_CONTI+ self.N_DISC*self.N_EMBED
        self.N_TOTAL_EMBED = self.N_DISC*self.N_EMBED

        self.sess = session
        
        self.dx = tf.placeholder(tf.int32, [None, self.N_DISC])
        self.cx = tf.placeholder(tf.float32, [None, self.N_CONTI])
        self.y  = tf.placeholder(tf.float32, [None, self.N_CLASS])
        self.lr = tf.placeholder(tf.float32, shape=())
        self.keep_prob = tf.placeholder(tf.float32)
        
#       wide part on categorical features
        self.dx_trans = tf.transpose( self.dx, [1,0] )
    
    
#         for i in range(self.N_DISC):
            
#             with tf.variable_scope("wide"+str(i)):
#                 w= tf.Variable(\
#                 tf.random_uniform([self.N_DISC_VOCA[i], self.N_CLASS],-1.0, 1.0) )
# #                         tf.random_normal([self.N_DISC_VOCA, self.N_CLASS],\
# #                         stddev=math.sqrt(2.0/float(self.N_DISC_VOCA))) )
#                 if i==0:
#                     dx_wsum = tf.nn.embedding_lookup( w, self.dx_trans[i] )
                    
# #                   L2
#                     self.regularizer_wide = tf.nn.l2_loss(w)
                    
#                 else:
#                     dx_wsum = dx_wsum + tf.nn.embedding_lookup( w, self.dx_trans[i] )
                    
# #                   L2  
#                     self.regularizer_wide = self.regularizer_wide + tf.nn.l2_loss(w)
               
    
#       co-occurrence of categorical features
        for i in range(20):
            
            tmpvoca1 = self.N_DISC_VOCA[i] 
            
            for j in range(i+1,20):
                
                tmpvoca2 = self.N_DISC_VOCA[j]
                
                with tf.variable_scope("cooccur"+str(i)+str(j)):
                    
                    
#                   vector approximate of co-occurrence matrix 
                    w1 = tf.Variable(tf.random_normal([self.N_CLASS, tmpvoca1, 1],\
                                            stddev=math.sqrt(2.0/float( tmpvoca1 ))) )
                    
                    w2 = tf.Variable(tf.random_normal([self.N_CLASS, 1, tmpvoca2],\
                                            stddev=math.sqrt(2.0/float( tmpvoca2 ))) )
            
                    b  = tf.Variable(tf.zeros( [ self.N_CLASS ] ))
                    
                    cooc_mat = []
                    for k in range(self.N_CLASS):
                        cooc_mat.append( tf.reshape( tf.matmul(w1[k],w2[k]), [-1,1] ) )
                        
#                   n_class by voca1*voca2  
                    cooc_vec = tf.stack(cooc_mat)
                    cooc_vec = tf.reshape(cooc_vec, [self.N_CLASS, -1] )
                    cooc_vec = tf.transpose(cooc_vec,[1,0])
                    
#                   build the idx of co-occurrence features
                    cooc_idx = self.dx_trans[i] * tmpvoca2 + self.dx_trans[j]
    
                    if i==0 and j==1:
#                       co-occurrence weighted sum
                        cooc_wsum  = tf.nn.embedding_lookup( cooc_vec, cooc_idx ) + b
#                       L2 
                        self.regularizer_cooc   = tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2) 
                    else:
                        cooc_wsum += tf.nn.embedding_lookup( cooc_vec, cooc_idx ) + b
#                       L2
                        self.regularizer_cooc += (tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2))
    
#       non-linear on co-occurrence weighted sum
        cooc_wsum = tf.nn.relu( cooc_wsum )
        
        
#       embedding of categorical features        
        tmp_embeded = []                
        for i in range(self.N_DISC):
            
            with tf.variable_scope("embedding"+str(i)):
                w= tf.Variable(\
                        tf.random_normal([self.N_DISC_VOCA[i], self.N_EMBED],\
                        stddev=math.sqrt(2.0/float(self.N_DISC_VOCA[i]))) )
#                     tf.random_uniform([self.N_DISC_VOCA, self.N_EMBED],-1.0, 1.0 )
                              
                tmp_embeded.append( tf.nn.embedding_lookup( w, self.dx_trans[i] ) )
                
        dx_embeded = tf.concat(tmp_embeded, 1)
        x_concate =  tf.concat( [self.cx,dx_embeded], 1 )
        
#       feature interaction
        for i in range(self.N_DISC):
            
            with tf.variable_scope("interact"+str(i)):
                w= tf.Variable(\
                        tf.random_normal([self.N_DISC_VOCA[i], self.N_CONTI],\
                        stddev=math.sqrt(2.0/float(self.N_DISC_VOCA[i]))) )
#                     tf.random_uniform([self.N_DISC_VOCA, self.N_EMBED],-1.0, 1.0 )
        
                if i==0:
                    inter_wsum  = tf.nn.embedding_lookup( w, self.dx_trans[i] )
#                   L2 
                    self.regularizer_inter   = tf.nn.l2_loss(w)
            
                else:
                    inter_wsum += tf.nn.embedding_lookup( w, self.dx_trans[i] )
#                   L2 
                    self.regularizer_inter += tf.nn.l2_loss(w)

    
#       add noise bias
        singleCol_h = tf.reduce_sum( self.cx * inter_wsum, 1   )
        tmp_h=[]
        for i in range( n_hidden_list[0] ):
            with tf.variable_scope("noise"+str( i )):
                noise_b = tf.Variable(tf.random_normal([ ],\
                        stddev=math.sqrt(2.0/float(n_hidden_list[0]))))
                tmp_h.append( singleCol_h + noise_b ) 
        
        
#         tmp_h=[]
#         for i in range( n_hidden_list[0] ):
#             with tf.variable_scope("noise"+str( i )):
#                 noise_b = tf.Variable(tf.random_normal([ self.N_CONTI ],\
#                         stddev=math.sqrt(2.0/float(n_hidden_list[0]))))
#                 tmp_h.append( tf.reduce_sum( self.cx * (inter_wsum + noise_b), 1 ) ) 
        
        
        tmp_h = tf.stack( tmp_h )
        tmp_h = tf.transpose( tmp_h, [1,0])
        
                 
#       interaction hidden layers 
        with tf.variable_scope("inter_h0"):
            
            w= tf.Variable(tf.random_normal([self.N_TOTAL, n_hidden_list[0]],\
                        stddev=math.sqrt(2.0/float(self.N_TOTAL)))) 
            b= tf.Variable(tf.zeros( [ n_hidden_list[0] ] ))
            
            h_inter = tf.matmul(x_concate, w)
            h_inter = h_inter + tmp_h
            h_inter = tf.nn.relu( tf.add( h_inter, b ) )
            
#           L2   
            self.regularizer = tf.nn.l2_loss(w)
        
#       dropout
#         h_inter = tf.nn.dropout(h_inter, self.keep_prob)
        
        for i in range(1, self.N_HIDDEN_LAYERS):
            
            with tf.variable_scope("inter_h"+str(i)):
                w = tf.Variable(\
                        tf.random_normal([n_hidden_list[i-1],n_hidden_list[i]],\
                                stddev=math.sqrt(2.0/float(n_hidden_list[i-1])))) 
                b = tf.Variable(tf.zeros( [n_hidden_list[i]] ))
                h_inter = tf.nn.relu( tf.add( tf.matmul(h_inter, w),b) )
        
#               L2
                self.regularizer += tf.nn.l2_loss(w)
        
        
#       output layer  
        h = h_inter
        
        with tf.variable_scope("output"):
            
            w = tf.Variable(tf.random_normal([n_hidden_list[self.N_HIDDEN_LAYERS-1],\
                                             self.N_CLASS],\
            stddev=math.sqrt(2.0/float(n_hidden_list[self.N_HIDDEN_LAYERS-1])))) 
            b = tf.Variable(tf.zeros( [ self.N_CLASS ] ))
            
            h = tf.matmul(h, w)
#             h = tf.add( h, dx_wsum )
            h = tf.add( h, cooc_wsum )
            h = tf.add( h, b )
        
            self.logit = h
            
#           L2  
            self.regularizer += tf.nn.l2_loss(w) 
    
#       overall L2
        self.regularizer += (self.regularizer_cooc + self.regularizer_inter)
#       self.regularizer_wide + 
         
        
#   initialize loss and optimization operations for training
    def train_ini(self):
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(\
                                                    logits=self.logit, labels=self.y) \
                                  + self.L2*self.regularizer )
        self.optimizer = \
        tf.train.AdadeltaOptimizer(learning_rate = self.LEARNING_RATE).minimize(self.cost)
#       tf.train.RMSPropOptimizer(learning_rate = self.LEARNING_RATE).minimize(self.cost)
#         tf.train.AdamOptimizer(learning_rate = self.lr).minimize(self.cost)
#         tf.train.RMSPropOptimizer(learning_rate = self.lr).minimize(self.cost)
#         tf.train.AdadeltaOptimizer(learning_rate = self.lr).minimize(self.cost)
#         tf.train.AdamOptimizer(learning_rate = self.lr).minimize(self.cost)
#         tf.train.GradientDescentOptimizer(learning_rate = self.lr).minimize(self.cost)
        
        self.init = tf.global_variables_initializer()
        sess.run( self.init )
        
#   training on batch of data
    def train_batch(self, dx_batch,cx_batch, y_batch, keep_prob, lr):
        
        _,c = sess.run([self.optimizer,self.cost],\
                       feed_dict={self.dx:dx_batch, \
                                  self.cx:cx_batch, self.y:y_batch,\
                                  self.lr:lr,\
                                  self.keep_prob:keep_prob\
                                 })
        
        return c
        
#   initialize inference         
    def inference_ini(self):
        self.correct_prediction = tf.equal(tf.argmax(self.logit,1), tf.argmax(self.y,1))
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, "float"))
        
#   infer givn testing data    
    def inference(self, dx_test, cx_test, y_test, keep_prob):
        return sess.run([self.accuracy], feed_dict={self.dx:dx_test,\
                                                  self.cx:cx_test, self.y:y_test,\
                                                  self.keep_prob:keep_prob\
                                                   })
        
#   unit_test
    def test(self, dx_test, cx_test, y_test ):
        self.init = tf.global_variables_initializer()
        sess.run( self.init )
        return sess.run( [self.tmp1], \
                 feed_dict={self.dx:dx_test, self.cx:cx_test, self.y:y_test})
        

In [4]:
para_n_epoch = 1200

# tunable parameters

#   representation ability
para_n_hidden_list = [ 32,32,16 ]
#     128,16,8
para_lr = 0.065
para_n_embedding = 2
#   regularization
para_batch_size = 32
para_keep_prob = 0.8
para_l2 = 0.25

# fixed parameters
para_n_class = 8
para_n_disc = 80
para_n_conti = 106
para_n_disc_voca = [8]*para_n_disc

# evaluation parameters
para_eval_byepoch = 10



with tf.Session() as sess:
    
    clf = InterNN_fuse( para_n_conti, para_n_disc, para_n_class, \
                      para_n_disc_voca, sess, para_n_embedding, para_n_hidden_list,\
                        para_lr, para_l2, para_batch_size)
    clf.train_ini()
    clf.inference_ini()
    
    para_cur_lr=para_lr
    
    total_cnt= np.shape(cxtrain)[0]
    total_batch = int(total_cnt/para_batch_size)
    
    total_idx=range(total_cnt)
    
#   training cycle
    for epoch in range(para_n_epoch):
        
        tmpc=0.0
        
#       shuffle traning instances each epoch  
        np.random.shuffle(total_idx)
    
#       Loop over all batches
        for i in range(total_batch):
            
            batch_idx = total_idx[ i*para_batch_size: (i+1)*para_batch_size ] 
            
            batch_cx = cxtrain[ batch_idx ]
            batch_dx = dxtrain[ batch_idx ]
            batch_y  =  ytrain[ batch_idx ]
            
            tmpc += clf.train_batch( batch_dx, batch_cx, batch_y, para_keep_prob,\
                                     para_cur_lr )
        
        if epoch%para_eval_byepoch != 0:
            continue
    
        tmp_test_acc  = clf.inference(dxtest, cxtest, ytest, para_keep_prob) 
        tmp_train_acc = clf.inference(dxtrain, cxtrain, ytrain, para_keep_prob) 
        
        print "loss on epoch ", epoch, " : ", 1.0*tmpc/total_batch, tmp_test_acc,\
        tmp_train_acc
    
    print "Optimization Finished!"

loss on epoch  0  :  2913.48150522 [0.10246433] [0.1140098]
loss on epoch  10  :  2008.20185174 [0.2386511] [0.24243298]
loss on epoch  20  :  1134.39717102 [0.22827497] [0.25194579]
loss on epoch  30  :  552.362633034 [0.23346303] [0.26722398]
loss on epoch  40  :  234.95337126 [0.27107653] [0.27947536]
loss on epoch  50  :  88.1163170779 [0.27367055] [0.27817816]
loss on epoch  60  :  30.3773007658 [0.2697795] [0.28552896]
loss on epoch  70  :  10.9335334698 [0.27885863] [0.28653792]
loss on epoch  80  :  5.02554154396 [0.29961088] [0.29071778]
loss on epoch  90  :  3.08871254215 [0.30609599] [0.29864514]
loss on epoch  100  :  2.27907398122 [0.31387809] [0.30570769]
loss on epoch  110  :  1.94261367012 [0.31647211] [0.30729318]
loss on epoch  120  :  1.83664976502 [0.31517509] [0.3116172]
loss on epoch  130  :  1.80286608416 [0.32036316] [0.31291437]
loss on epoch  140  :  1.78456078304 [0.32166019] [0.31651774]
loss on epoch  150  :  1.77202774253 [0.32555124] [0.31723839]
loss on 

ERROR: Internal Python error in the inspect module.
Below is the traceback from this internal error.



  File "/usr/lib/python2.7/inspect.py", line 1004, in getframeinfo
    filename = getsourcefile(frame) or getfile(frame)
  File "/usr/lib/python2.7/inspect.py", line 454, in getsourcefile
    if hasattr(getmodule(object, filename), '__loader__'):
  File "/usr/lib/python2.7/inspect.py", line 490, in getmodule
    for modname, module in sys.modules.items():
KeyboardInterrupt



Unfortunately, your original traceback can not be constructed.



IndexError: string index out of range

In [ ]:
[ 128,64,16 ]
0.06 0.35

0.05, 0.35
loss on epoch  0  :  4112.37932897 [0.1608301] [0.13995388]
loss on epoch  10  :  3059.09017605 [0.25680932] [0.23681176]
loss on epoch  20  :  1960.28582029 [0.25810635] [0.25295475]
loss on epoch  30  :  1124.10197449 [0.28145266] [0.2711156]
loss on epoch  40  :  583.568243804 [0.30998704] [0.27313346]
loss on epoch  50  :  274.963851929 [0.27367055] [0.27990776]
loss on epoch  60  :  117.810368503 [0.27496758] [0.27385414]
loss on epoch  70  :  46.1854696627 [0.28534371] [0.28077257]
loss on epoch  80  :  16.9772326946 [0.29053178] [0.28942057]
loss on epoch  90  :  6.35192130451 [0.29831389] [0.30138367]
loss on epoch  100  :  3.05701557464 [0.30869001] [0.31392333]
loss on epoch  110  :  2.1787649371 [0.31776914] [0.31954452]
loss on epoch  120  :  1.93165977023 [0.31647211] [0.32718363]
loss on epoch  130  :  1.84372538659 [0.31906614] [0.32992217]
loss on epoch  140  :  1.80727599948 [0.31258106] [0.33395791]
loss on epoch  150  :  1.78846734707 [0.31128404] [0.33611992]
loss on epoch  160  :  1.77639710903 [0.31647211] [0.33568752]
loss on epoch  170  :  1.76785224787 [0.31517509] [0.33900261]
loss on epoch  180  :  1.76091153037 [0.31906614] [0.33972326]
loss on epoch  190  :  1.7558026805 [0.31387809] [0.34361488]
loss on epoch  200  :  1.75204727881 [0.31258106] [0.34592101]
loss on epoch  210  :  1.74816672449 [0.31387809] [0.34779474]
loss on epoch  220  :  1.74480803035 [0.31387809] [0.35240704]
loss on epoch  230  :  1.74340877875 [0.31517509] [0.35298356]
loss on epoch  240  :  1.74105864929 [0.31906614] [0.3561545]
loss on epoch  250  :  1.73833712273 [0.31906614] [0.3557221]
loss on epoch  260  :  1.73768513843 [0.31517509] [0.35428077]
loss on epoch  270  :  1.736201368 [0.31647211] [0.35557798]
loss on epoch  280  :  1.73401567285 [0.31258106] [0.3561545]
loss on epoch  290  :  1.731798406 [0.31387809] [0.36292881]
loss on epoch  300  :  1.72962650546 [0.32036316] [0.36091092]
loss on epoch  310  :  1.72767726249 [0.32036316] [0.3672528]
loss on epoch  320  :  1.72687107545 [0.31517509] [0.3676852]
loss on epoch  330  :  1.72526504706 [0.31258106] [0.36869416]
loss on epoch  340  :  1.72467513493 [0.31128404] [0.36999136]
loss on epoch  350  :  1.72244894118 [0.31128404] [0.37546843]
loss on epoch  360  :  1.7203446726 [0.30998704] [0.37561256]
loss on epoch  370  :  1.7189220477 [0.32295719] [0.37849525]
loss on epoch  380  :  1.71831009951 [0.31387809] [0.37791872]
loss on epoch  390  :  1.71590292178 [0.31647211] [0.38209859]
loss on epoch  400  :  1.71552342728 [0.31387809] [0.39017007]


[ 128,32,16 ]

0.05 0.25
loss on epoch  0  :  2984.16321818 [0.14526589] [0.14600749]
loss on epoch  10  :  2228.49032367 [0.21011673] [0.23493803]
loss on epoch  20  :  1435.26218725 [0.22568093] [0.23839723]
loss on epoch  30  :  827.165083709 [0.24383917] [0.25756702]
loss on epoch  40  :  432.92500616 [0.27626458] [0.2826463]
loss on epoch  50  :  206.896535732 [0.28145266] [0.28999713]
loss on epoch  60  :  91.1961491903 [0.29182878] [0.294177]
loss on epoch  70  :  38.1252215527 [0.29312581] [0.29734793]
loss on epoch  80  :  16.1940223685 [0.29571983] [0.29922166]
loss on epoch  90  :  7.80226797969 [0.31647211] [0.30945519]
loss on epoch  100  :  4.63881570101 [0.32425421] [0.3185356]
loss on epoch  110  :  3.2436019403 [0.32036316] [0.32905737]
loss on epoch  120  :  2.52279345194 [0.31258106] [0.33972326]
loss on epoch  130  :  2.141223509 [0.31387809] [0.34476796]
loss on epoch  140  :  1.95561428699 [0.31517509] [0.35802826]
loss on epoch  150  :  1.86308251487 [0.31776914] [0.36754107]
loss on epoch  160  :  1.81187883516 [0.31258106] [0.37590083]
loss on epoch  170  :  1.78083807672 [0.30869001] [0.38887286]
loss on epoch  180  :  1.75611915246 [0.31258106] [0.3979533]
loss on epoch  190  :  1.73424496474 [0.31517509] [0.41164601]
loss on epoch  200  :  1.71701375809 [0.31776914] [0.42332083]
loss on epoch  210  :  1.6997012634 [0.30739298] [0.43441916]
loss on epoch  220  :  1.68647382436 [0.28145266] [0.43773422]
loss on epoch  230  :  1.67338317412 [0.28404668] [0.44767946]
loss on epoch  240  :  1.66195621259 [0.27885863] [0.45503026]
loss on epoch  250  :  1.65262653982 [0.28664073] [0.46036321]
loss on epoch  260  :  1.64566449987 [0.28404668] [0.4628135]
loss on epoch  270  :  1.63894850071 [0.29053178] [0.48255983]
loss on epoch  280  :  1.6328404325 [0.2775616] [0.47852406]
loss on epoch  290  :  1.62758091368 [0.27885863] [0.48385701]
loss on epoch  300  :  1.62190970337 [0.28404668] [0.48861343]

0.75
loss on epoch  0  :  2977.35766489 [0.16990921] [0.14946671]
loss on epoch  10  :  2217.23882152 [0.22568093] [0.21230902]
loss on epoch  20  :  1420.06968067 [0.23605707] [0.24185644]
loss on epoch  30  :  815.662684123 [0.24773023] [0.24646872]
loss on epoch  40  :  424.706616154 [0.25551233] [0.25295475]
loss on epoch  50  :  201.05387391 [0.25291827] [0.24762179]
loss on epoch  60  :  86.85074068 [0.25291827] [0.24848659]
loss on epoch  70  :  34.9947918609 [0.25680932] [0.24661286]
loss on epoch  80  :  13.9666180081 [0.25551233] [0.24603632]
loss on epoch  90  :  6.26568315427 [0.25551233] [0.24589218]
loss on epoch  100  :  3.58276599535 [0.31517509] [0.27976361]
loss on epoch  110  :  2.52540745007 [0.31258106] [0.28596136]
loss on epoch  120  :  2.00819005845 [0.31776914] [0.29691553]
loss on epoch  130  :  1.82808638668 [0.31387809] [0.29965407]
loss on epoch  140  :  1.77833122843 [0.30998704] [0.30325744]
loss on epoch  150  :  1.75744953089 [0.31128404] [0.30844623]
loss on epoch  160  :  1.74410498749 [0.31387809] [0.31248197]
loss on epoch  170  :  1.73372809147 [0.31387809] [0.318968]
loss on epoch  180  :  1.72589197865 [0.31258106] [0.32257134]
loss on epoch  190  :  1.71831875101 [0.31128404] [0.32992217]
loss on epoch  200  :  1.71030590214 [0.31128404] [0.33655232]
loss on epoch  210  :  1.70321574145 [0.31258106] [0.34145287]
loss on epoch  220  :  1.69467934249 [0.30220494] [0.34779474]
loss on epoch  230  :  1.68479432828 [0.30479896] [0.36523494]
loss on epoch  240  :  1.67157381994 [0.29442284] [0.38166618]
loss on epoch  250  :  1.6562432128 [0.30220494] [0.40386277]
loss on epoch  260  :  1.64067010195 [0.28793773] [0.42288843]
loss on epoch  270  :  1.6255188049 [0.28404668] [0.43946382]
loss on epoch  280  :  1.61254157347 [0.28274968] [0.45675987]
loss on epoch  290  :  1.6003067085 [0.27626458] [0.48054194]
loss on epoch  300  :  1.58969467106 [0.28664073] [0.50446814]
loss on epoch  310  :  1.58066189786 [0.28534371] [0.5174402]
loss on epoch  320  :  1.57223566704 [0.27885863] [0.53343904]
loss on epoch  330  :  1.5658605165 [0.28145266] [0.55736524]
loss on epoch  340  :  1.56251320188 [0.27237353] [0.58475065]
loss on epoch  350  :  1.55729498907 [0.27107653] [0.58878642]
loss on epoch  360  :  1.55314406201 [0.27367055] [0.59613723]
loss on epoch  370  :  1.54791894114 [0.26848248] [0.60348803]
loss on epoch  380  :  1.54467611125 [0.27107653] [0.61098301]
loss on epoch  390  :  1.54223347428 [0.26070037] [0.61804557]
loss on epoch  400  :  1.53922743323 [0.26718548] [0.62063998]
loss on epoch  410  :  1.53627740564 [0.27237353] [0.62654942]
loss on epoch  420  :  1.53325057526 [0.27367055] [0.62568462]
loss on epoch  430  :  1.53148831703 [0.25940338] [0.62741423]
loss on epoch  440  :  1.53023800419 [0.28274968] [0.62251371]
loss on epoch  450  :  1.5295211728 [0.26718548] [0.63245893]
loss on epoch  460  :  1.52626908101 [0.2619974] [0.63390028]
loss on epoch  470  :  1.52444877393 [0.26070037] [0.63562989]
loss on epoch  480  :  1.52219186999 [0.27107653] [0.63851255]
loss on epoch  490  :  1.52172864863 [0.26848248] [0.637936]
loss on epoch  500  :  1.52001977464 [0.26848248] [0.63851255]
loss on epoch  510  :  1.51810660462 [0.27626458] [0.62972039]
loss on epoch  520  :  1.51779708266 [0.27367055] [0.64053041]
loss on epoch  530  :  1.5144958899 [0.26459143] [0.6396656]
loss on epoch  540  :  1.51424664811 [0.26718548] [0.63735944]

[ 32,32,16 ]: #neurons, regularization

0.75 0.25   

loss on epoch  0  :  2937.00183784 [0.13748379] [0.16431248]
loss on epoch  10  :  1980.56166077 [0.23476005] [0.22686653]
loss on epoch  20  :  1060.14993936 [0.22178988] [0.23133467]
loss on epoch  30  :  469.47508932 [0.20622568] [0.2382531]
loss on epoch  40  :  173.987235387 [0.23994812] [0.24791007]
loss on epoch  50  :  54.8401287927 [0.25551233] [0.24661286]
loss on epoch  60  :  15.6860910345 [0.25551233] [0.24589218]
loss on epoch  70  :  5.05427253909 [0.25551233] [0.24589218]
loss on epoch  80  :  2.48936366483 [0.25551233] [0.24589218]
loss on epoch  90  :  1.92046676135 [0.25551233] [0.24589218]
loss on epoch  100  :  1.81257838894 [0.25551233] [0.24589218]
loss on epoch  110  :  1.79125653152 [0.2697795] [0.25626981]
loss on epoch  120  :  1.78011440182 [0.2697795] [0.25713462]
loss on epoch  130  :  1.77279601826 [0.26459143] [0.25843182]
loss on epoch  140  :  1.76641532116 [0.26848248] [0.26535025]
loss on epoch  150  :  1.76214704359 [0.26718548] [0.26924187]
loss on epoch  160  :  1.75710989204 [0.26848248] [0.27082732]
loss on epoch  170  :  1.75269452731 [0.26588845] [0.27443066]
loss on epoch  180  :  1.74956474978 [0.31387809] [0.30023062]
loss on epoch  190  :  1.74727627469 [0.31517509] [0.30556357]
loss on epoch  200  :  1.74427712129 [0.32555124] [0.30758142]
loss on epoch  210  :  1.74187214673 [0.31906614] [0.30988759]
loss on epoch  220  :  1.73836385597 [0.31776914] [0.30916691]
loss on epoch  230  :  1.73654806172 [0.32555124] [0.31377918]
loss on epoch  240  :  1.73394887701 [0.32166019] [0.31680599]
loss on epoch  250  :  1.7322146302 [0.33073929] [0.31954452]
loss on epoch  260  :  1.73029606651 [0.32814527] [0.32127413]
loss on epoch  270  :  1.72917312604 [0.33463034] [0.32386854]
loss on epoch  280  :  1.72733631344 [0.32425421] [0.32559815]
loss on epoch  290  :  1.72618572745 [0.32814527] [0.32603055]
loss on epoch  300  :  1.72489549772 [0.32684824] [0.32516575]
loss on epoch  310  :  1.72346471581 [0.32555124] [0.32747188]
loss on epoch  320  :  1.72210429664 [0.32555124] [0.32703948]
loss on epoch  330  :  1.7202126737 [0.32166019] [0.33366963]
loss on epoch  340  :  1.7197007432 [0.32166019] [0.33208418]
loss on epoch  350  :  1.71754157102 [0.31387809] [0.3377054]
loss on epoch  360  :  1.71545168206 [0.31776914] [0.34001154]
loss on epoch  370  :  1.7137490522 [0.31387809] [0.34505621]
loss on epoch  380  :  1.7121563013 [0.31906614] [0.34779474]
loss on epoch  390  :  1.70994089654 [0.31128404] [0.35413665]
loss on epoch  400  :  1.70707785145 [0.31647211] [0.3565869]
loss on epoch  410  :  1.70359759695 [0.31387809] [0.36710867]
loss on epoch  420  :  1.69967268959 [0.31258106] [0.37518016]
loss on epoch  430  :  1.69559655421 [0.29961088] [0.38267511]
loss on epoch  440  :  1.69131073908 [0.30739298] [0.40458345]
loss on epoch  450  :  1.68664100932 [0.29442284] [0.41669068]
loss on epoch  460  :  1.6815467957 [0.29961088] [0.43816662]
loss on epoch  470  :  1.67764298673 [0.29571983] [0.45575094]
    
0.65 0.25

loss on epoch  0  :  2886.24161219 [0.19455253] [0.20351687]
loss on epoch  10  :  1988.59360532 [0.1997406] [0.22383973]
loss on epoch  20  :  1128.57974187 [0.24383917] [0.25381956]
loss on epoch  30  :  552.771804244 [0.25551233] [0.26535025]
loss on epoch  40  :  237.863297639 [0.26588845] [0.28048429]
loss on epoch  50  :  91.2934027071 [0.25810635] [0.27961949]
loss on epoch  60  :  32.7627919515 [0.27237353] [0.29143846]
loss on epoch  70  :  12.5326693058 [0.28404668] [0.29806861]
loss on epoch  80  :  6.06290647719 [0.30609599] [0.31522053]
loss on epoch  90  :  3.9064360857 [0.30479896] [0.33309311]
loss on epoch  100  :  2.95455121663 [0.30998704] [0.33539924]
loss on epoch  110  :  2.42096429291 [0.31906614] [0.34808302]
loss on epoch  120  :  2.10320025738 [0.31776914] [0.35154223]
loss on epoch  130  :  1.92841164105 [0.31776914] [0.34678581]
loss on epoch  140  :  1.83906269681 [0.32036316] [0.34246179]
loss on epoch  150  :  1.79554319271 [0.32814527] [0.34058806]
loss on epoch  160  :  1.77310961043 [0.32555124] [0.34073219]
loss on epoch  170  :  1.76078001603 [0.32555124] [0.33799365]
loss on epoch  180  :  1.75251507538 [0.32425421] [0.33885846]
loss on epoch  190  :  1.74655083208 [0.31776914] [0.33511099]
loss on epoch  200  :  1.74213193688 [0.32425421] [0.33712885]
loss on epoch  210  :  1.73741377283 [0.32684824] [0.3368406]
loss on epoch  220  :  1.73437432503 [0.32295719] [0.33943498]
loss on epoch  230  :  1.73158523275 [0.31776914] [0.34001154]
loss on epoch  240  :  1.72955057466 [0.31387809] [0.33986738]
loss on epoch  250  :  1.72650755666 [0.31776914] [0.3428942]
loss on epoch  260  :  1.72500768745 [0.31647211] [0.34174114]
loss on epoch  270  :  1.72295075103 [0.31258106] [0.34303835]
loss on epoch  280  :  1.72162079866 [0.31258106] [0.34260595]
loss on epoch  290  :  1.71957002415 [0.31906614] [0.34505621]
loss on epoch  300  :  1.71809021577 [0.31906614] [0.34505621]
loss on epoch  310  :  1.71719058575 [0.32166019] [0.3446238]
loss on epoch  320  :  1.71635875051 [0.31776914] [0.3441914]
loss on epoch  330  :  1.71525748847 [0.31517509] [0.34520036]
loss on epoch  340  :  1.71390254244 [0.31387809] [0.34606513]
loss on epoch  350  :  1.71252528109 [0.31387809] [0.34606513]
loss on epoch  360  :  1.71242676675 [0.31776914] [0.34851542]
loss on epoch  370  :  1.71220593486 [0.31906614] [0.34779474]
loss on epoch  380  :  1.71055379841 [0.31776914] [0.35110983]
loss on epoch  390  :  1.71012227348 [0.31517509] [0.35110983]
loss on epoch  400  :  1.7088378723 [0.31647211] [0.34909195]
loss on epoch  410  :  1.70774036867 [0.31776914] [0.35096571]
loss on epoch  420  :  1.70759297245 [0.31647211] [0.35211876]
loss on epoch  430  :  1.70742613629 [0.31128404] [0.35110983]
loss on epoch  440  :  1.70667638602 [0.31387809] [0.35269532]
loss on epoch  450  :  1.70597999405 [0.31387809] [0.35154223]
loss on epoch  460  :  1.70544632441 [0.30609599] [0.35139811]
loss on epoch  470  :  1.70489510232 [0.30739298] [0.35298356]
loss on epoch  480  :  1.70388288796 [0.31517509] [0.35312769]
loss on epoch  490  :  1.70463823444 [0.30739298] [0.35211876]
loss on epoch  500  :  1.70427047489 [0.30869001] [0.35327184]
loss on epoch  510  :  1.70289952943 [0.30998704] [0.35471317]
loss on epoch  520  :  1.70337097678 [0.31387809] [0.35442489]
loss on epoch  530  :  1.70273655598 [0.31647211] [0.3552897]
loss on epoch  540  :  1.70253641407 [0.31776914] [0.3548573]
loss on epoch  550  :  1.70112308418 [0.31387809] [0.3557221]
loss on epoch  560  :  1.70103886889 [0.31258106] [0.3552897]
loss on epoch  570  :  1.70150572282 [0.31647211] [0.35586625]
loss on epoch  580  :  1.7008207926 [0.30739298] [0.35773998]
loss on epoch  590  :  1.6999411307 [0.30869001] [0.35759586]
loss on epoch  600  :  1.69985376298 [0.30869001] [0.35817239]
loss on epoch  610  :  1.6987061197 [0.31258106] [0.36134332]
loss on epoch  620  :  1.69947070601 [0.30869001] [0.36091092]
loss on epoch  630  :  1.69928016872 [0.30998704] [0.35990199]
loss on epoch  640  :  1.69800751794 [0.31517509] [0.35961372]
loss on epoch  650  :  1.698172896 [0.31517509] [0.3616316]
loss on epoch  660  :  1.69777368727 [0.31128404] [0.36278465]
loss on epoch  670  :  1.69761315982 [0.30350193] [0.3603344]
loss on epoch  680  :  1.69764046261 [0.31647211] [0.36408186]
loss on epoch  690  :  1.69709036085 [0.30869001] [0.36220813]
loss on epoch  700  :  1.69737868232 [0.30739298] [0.36321706]
loss on epoch  710  :  1.69611418026 [0.31258106] [0.36408186]
loss on epoch  720  :  1.69624953745 [0.30739298] [0.36148745]
loss on epoch  730  :  1.69619914613 [0.30739298] [0.36696455]
loss on epoch  740  :  1.69532852868 [0.31128404] [0.36393774]
loss on epoch  750  :  1.69535909262 [0.30609599] [0.36595562]
loss on epoch  760  :  1.69589098515 [0.30998704] [0.36739695]
loss on epoch  770  :  1.69526974139 [0.30350193] [0.36754107]
loss on epoch  780  :  1.69432959788 [0.31128404] [0.36710867]
loss on epoch  790  :  1.69514299819 [0.31128404] [0.36883828]
loss on epoch  800  :  1.69410144013 [0.31128404] [0.37013549]
loss on epoch  810  :  1.69474113319 [0.30739298] [0.36826175]
loss on epoch  820  :  1.6939667811 [0.30479896] [0.36782935]
loss on epoch  830  :  1.69408524588 [0.31387809] [0.36883828]
loss on epoch  840  :  1.69354515992 [0.31387809] [0.37186509]
loss on epoch 
    
0.55 no dropout

loss on epoch  0  :  2874.94279197 [0.1608301] [0.16070914]
loss on epoch  10  :  2083.56454524 [0.23605707] [0.22902854]
loss on epoch  20  :  1281.37241618 [0.25162128] [0.24531566]
loss on epoch  30  :  698.150010851 [0.24902724] [0.25036034]
loss on epoch  40  :  341.334552765 [0.27107653] [0.2660709]
loss on epoch  50  :  150.811489388 [0.28664073] [0.27486306]
loss on epoch  60  :  60.9060257982 [0.28015563] [0.28524071]
loss on epoch  70  :  23.8454049075 [0.27626458] [0.28610551]
loss on epoch  80  :  10.3388724415 [0.28534371] [0.29201499]
loss on epoch  90  :  5.56707937188 [0.29053178] [0.29259151]
loss on epoch  100  :  3.68248138935 [0.27367055] [0.28855577]
loss on epoch  110  :  2.76654098762 [0.25551233] [0.28624964]
loss on epoch  120  :  2.27717612849 [0.25291827] [0.27889881]
loss on epoch  130  :  2.00933238864 [0.24254215] [0.26347652]
loss on epoch  140  :  1.89256767542 [0.24124514] [0.26318824]
loss on epoch  150  :  1.84241652875 [0.2386511] [0.25900835]
loss on epoch  160  :  1.81760840835 [0.2386511] [0.2523782]
loss on epoch  170  :  1.8034652168 [0.24124514] [0.2493514]
loss on epoch  180  :  1.79319385191 [0.25032425] [0.24329779]
loss on epoch  190  :  1.7860051239 [0.23994812] [0.24430671]
loss on epoch  200  :  1.7799572183 [0.23605707] [0.23969443]
loss on epoch  210  :  1.77461905777 [0.24773023] [0.24041511]
loss on epoch  220  :  1.77011702955 [0.24513619] [0.24430671]
loss on epoch  230  :  1.76598942335 [0.25551233] [0.24589218]
loss on epoch  240  :  1.76272176316 [0.25551233] [0.24589218]
loss on epoch  250  :  1.75966795837 [0.25551233] [0.24589218]
loss on epoch  260  :  1.75665102126 [0.25551233] [0.24589218]
loss on epoch  270  :  1.75511636006 [0.25551233] [0.24589218]
loss on epoch  280  :  1.75301427532 [0.25551233] [0.24589218]
loss on epoch  290  :  1.75122346646 [0.25551233] [0.24589218]
loss on epoch  300  :  1.74980408229 [0.25551233] [0.24589218]
loss on epoch  310  :  1.74853351381 [0.29961088] [0.27515134]
loss on epoch  320  :  1.74744045679 [0.30609599] [0.27716923]
loss on epoch  330  :  1.74594371849 [0.30869001] [0.28192562]
loss on epoch  340  :  1.74501032299 [0.31128404] [0.28452003]
loss on epoch  350  :  1.74359217728 [0.31517509] [0.28725857]
loss on epoch  360  :  1.74254825822 [0.32166019] [0.28999713]
loss on epoch  370  :  1.7404457281 [0.32036316] [0.29388872]
loss on epoch  380  :  1.73958398126 [0.32684824] [0.29662728]
loss on epoch  390  :  1.73755385368 [0.31776914] [0.29878926]
loss on epoch  400  :  1.73575710643 [0.31906614] [0.30196023]
loss on epoch  410  :  1.73401501775 [0.32814527] [0.30628422]
loss on epoch  420  :  1.73316884703 [0.32425421] [0.31003171]
loss on epoch  430  :  1.73088525291 [0.32295719] [0.31406745]
loss on epoch  440  :  1.72995901494 [0.33073929] [0.31666186]
loss on epoch  450  :  1.72873910599 [0.32814527] [0.31666186]
loss on epoch  460  :  1.72727826348 [0.32814527] [0.31579706]
loss on epoch  470  :  1.7261250858 [0.32814527] [0.32069761]
loss on epoch  480  :  1.7253982938 [0.32684824] [0.32069761]
loss on epoch  490  :  1.72430204573 [0.32684824] [0.32213894]
loss on epoch  500  :  1.72306088496 [0.32814527] [0.3253099]
loss on epoch  510  :  1.72182659418 [0.32944229] [0.32689536]
loss on epoch  520  :  1.7212175026 [0.32814527] [0.32703948]
loss on epoch  530  :  1.72045994578 [0.33203632] [0.32833669]
loss on epoch  540  :  1.71956506096 [0.32555124] [0.33237243]
loss on epoch  550  :  1.7187097917 [0.32814527] [0.3322283]
loss on epoch  560  :  1.7172617101 [0.32555124] [0.3359758]
loss on epoch  570  :  1.71600874927 [0.31258106] [0.33669645]
loss on epoch  580  :  1.71367786438 [0.31517509] [0.34015566]
loss on epoch  590  :  1.71271143026 [0.32295719] [0.33756125]
loss on epoch  600  :  1.71196182734 [0.31387809] [0.34087634]
loss on epoch  610  :  1.71009510038 [0.31387809] [0.34808302]
loss on epoch  620  :  1.70871090337 [0.30998704] [0.34981263]
loss on epoch  630  :  1.70718104806 [0.31776914] [0.35384837]
loss on epoch  640  :  1.7056134023 [0.31776914] [0.3552897]
loss on epoch  650  :  1.7038799149 [0.31647211] [0.36278465]
loss on epoch  660  :  1.70147576045 [0.30998704] [0.36465842]
loss on epoch  670  :  1.69909681214 [0.31517509] [0.37013549]
loss on epoch  680  :  1.69627010602 [0.31128404] [0.37806284]
loss on epoch  690  :  1.69379743547 [0.31387809] [0.38685501]
loss on epoch  700  :  1.69017124949 [0.30998704] [0.38901702]
loss on epoch  710  :  1.68774709989 [0.31906614] [0.40357453]
loss on epoch  720  :  1.68479633607 [0.30998704] [0.40847507]
loss on epoch  730  :  1.68057860876 [0.29701686] [0.42476219]
loss on epoch  740  :  1.67775793042 [0.29571983] [0.43081579]
loss on epoch  750  :  1.67421361583 [0.29312581] [0.44292304]
loss on epoch  760  :  1.66988071192 [0.29571983] [0.44854426]


    
0.45 no dropout

loss on epoch  0  :  2904.77315267 [0.19714656] [0.18895936]
loss on epoch  10  :  2239.2465583 [0.21141374] [0.21533583]
loss on epoch  20  :  1507.24788694 [0.230869] [0.24113578]
loss on epoch  30  :  919.092349017 [0.24513619] [0.2541078]
loss on epoch  40  :  513.960989917 [0.25551233] [0.25799942]
loss on epoch  50  :  264.626757163 [0.2542153] [0.25425196]
loss on epoch  60  :  126.197195512 [0.24773023] [0.25309888]
loss on epoch  70  :  56.272663452 [0.24773023] [0.25468436]
loss on epoch  80  :  24.2569229161 [0.24513619] [0.26261172]
loss on epoch  90  :  10.8478422783 [0.2464332] [0.26679158]
loss on epoch  100  :  5.65829826947 [0.2464332] [0.27226865]
loss on epoch  110  :  3.68620026774 [0.25162128] [0.27543962]
loss on epoch  120  :  2.85285171535 [0.25940338] [0.27659267]
loss on epoch  130  :  2.42323571554 [0.26329443] [0.27270105]
loss on epoch  140  :  2.17573743231 [0.26588845] [0.27284521]
loss on epoch  150  :  2.0301572537 [0.26718548] [0.2702508]
loss on epoch  160  :  1.94258512888 [0.26459143] [0.27342173]
loss on epoch  170  :  1.88481268949 [0.2697795] [0.27716923]
loss on epoch  180  :  1.84617088845 [0.27107653] [0.28106081]
loss on epoch  190  :  1.81930811924 [0.2697795] [0.28178149]
loss on epoch  200  :  1.80086447188 [0.26718548] [0.28423178]
loss on epoch  210  :  1.7875992137 [0.32036316] [0.30859038]
loss on epoch  220  :  1.77780386474 [0.32684824] [0.31565294]
loss on epoch  230  :  1.76974088947 [0.32555124] [0.32040933]
loss on epoch  240  :  1.7632859283 [0.32295719] [0.32170653]
loss on epoch  250  :  1.75851491149 [0.32036316] [0.32213894]
loss on epoch  260  :  1.75423782402 [0.32295719] [0.32372442]
loss on epoch  270  :  1.75089333234 [0.32944229] [0.32473335]
loss on epoch  280  :  1.74837678836 [0.32684824] [0.32516575]
loss on epoch  290  :  1.74513709545 [0.32814527] [0.32516575]
loss on epoch  300  :  1.74362069755 [0.33073929] [0.32631883]
loss on epoch  310  :  1.74140824046 [0.32036316] [0.32660708]
loss on epoch  320  :  1.73898471064 [0.31906614] [0.32732776]
loss on epoch  330  :  1.73712137341 [0.32166019] [0.32992217]
loss on epoch  340  :  1.7356525207 [0.32425421] [0.33237243]
loss on epoch  350  :  1.73420486185 [0.32555124] [0.33208418]
loss on epoch  360  :  1.73237812188 [0.32555124] [0.3317959]
loss on epoch  370  :  1.7308102366 [0.32425421] [0.33309311]
loss on epoch  380  :  1.72947029383 [0.32814527] [0.33712885]
loss on epoch  390  :  1.72876107362 [0.32944229] [0.33611992]
loss on epoch  400  :  1.72705303133 [0.32684824] [0.337273]
loss on epoch  410  :  1.72517795254 [0.32166019] [0.33828193]
loss on epoch  420  :  1.72473206951 [0.31906614] [0.34231767]
loss on epoch  430  :  1.72311593703 [0.32295719] [0.34260595]
loss on epoch  440  :  1.72156693207 [0.31906614] [0.34447968]
loss on epoch  450  :  1.72039219958 [0.31776914] [0.34851542]
loss on epoch  460  :  1.71828975887 [0.32295719] [0.35125396]
loss on epoch  470  :  1.7153291465 [0.32295719] [0.35269532]
loss on epoch  480  :  1.71169379354 [0.31776914] [0.35932547]
loss on epoch  490  :  1.70989083913 [0.32036316] [0.36739695]
loss on epoch  500  :  1.7066930996 [0.31776914] [0.3727299]
loss on epoch  510  :  1.70284286583 [0.31776914] [0.38353992]
loss on epoch  520  :  1.69862202583 [0.32036316] [0.39680022]
loss on epoch  530  :  1.69522329889 [0.32295719] [0.40386277]
loss on epoch  540  :  1.69030124115 [0.30350193] [0.41741136]
loss on epoch  550  :  1.68574391637 [0.30479896] [0.42375323]
loss on epoch  560  :  1.68108665722 [0.29701686] [0.42923033]
loss on epoch  570  :  1.6782276647 [0.29053178] [0.44739118]



    
0.45 dropout    
loss on epoch  0  :  2337.22378088 [0.15953307] [0.15162872]
loss on epoch  10  :  1889.34066999 [0.22438392] [0.19789565]
loss on epoch  20  :  1397.70430897 [0.22697794] [0.21029115]
loss on epoch  30  :  975.793204696 [0.21271077] [0.2164889]
loss on epoch  40  :  657.499851086 [0.22827497] [0.23234361]
loss on epoch  50  :  433.534762488 [0.25032425] [0.24387431]
loss on epoch  60  :  283.06589282 [0.23994812] [0.23609109]
loss on epoch  70  :  186.267238617 [0.25291827] [0.23508215]
loss on epoch  80  :  125.115274783 [0.23735408] [0.23464975]
loss on epoch  90  :  86.2245288425 [0.25940338] [0.23796484]
loss on epoch  100  :  60.8256910289 [0.21789883] [0.20683195]
loss on epoch  110  :  43.7591062299 [0.26070037] [0.24300951]
loss on epoch  120  :  31.7362976163 [0.22438392] [0.22614586]
loss on epoch  130  :  22.9795110579 [0.22568093] [0.23046988]
loss on epoch  140  :  16.7677883131 [0.21789883] [0.22917268]
loss on epoch  150  :  12.3121599665 [0.23476005] [0.23392908]
loss on epoch  160  :  9.14334391223 [0.26070037] [0.25540501]
loss on epoch  170  :  6.85844402622 [0.27496758] [0.26203516]
loss on epoch  180  :  5.16672201731 [0.28534371] [0.27125973]
loss on epoch  190  :  4.00481683568 [0.29831389] [0.27601615]
loss on epoch  200  :  3.21034482011 [0.29571983] [0.2822139]
loss on epoch  210  :  2.70275750204 [0.29961088] [0.28927645]
loss on epoch  220  :  2.37622975972 [0.30479896] [0.2946094]
loss on epoch  230  :  2.1668948487 [0.30350193] [0.30988759]
loss on epoch  240  :  2.03203844527 [0.30739298] [0.31392333]
loss on epoch  250  :  1.94201788472 [0.31128404] [0.32761604]
loss on epoch  260  :  1.88699813353 [0.31647211] [0.33309311]
loss on epoch  270  :  1.84561692509 [0.31776914] [0.34001154]
loss on epoch  280  :  1.81618379553 [0.31387809] [0.34894782]
loss on epoch  290  :  1.79266019552 [0.31128404] [0.35788411]
loss on epoch  300  :  1.77263346977 [0.31906614] [0.36638802]
loss on epoch  310  :  1.7528932939 [0.31906614] [0.37172097]
loss on epoch  320  :  1.73470119746 [0.32166019] [0.39391756]
loss on epoch  330  :  1.71242918129 [0.31517509] [0.42173538]
loss on epoch  340  :  1.69382170781 [0.31647211] [0.43456328]
loss on epoch  350  :  1.67667607301 [0.30479896] [0.46007496]
loss on epoch  360  :  1.66357970734 [0.29831389] [0.48962238]
loss on epoch  370  :  1.65094748581 [0.29961088] [0.50763911]
loss on epoch  380  :  1.64171585827 [0.29571983] [0.5289709]
loss on epoch  390  :  1.63375818564 [0.29312581] [0.53963679]
loss on epoch  400  :  1.62783267542 [0.28793773] [0.5478524]



In [ ]:
[128,64]

loss on epoch  0  :  1210.32822616 [0.14526589] [0.15321419]
loss on epoch  10  :  975.476586801 [0.19844358] [0.20798501]
loss on epoch  20  :  710.190851847 [0.230869] [0.24286538]
loss on epoch  30  :  478.653611501 [0.24254215] [0.24401845]
loss on epoch  40  :  303.751033218 [0.24902724] [0.2541078]
loss on epoch  50  :  184.012676451 [0.2542153] [0.26333237]
loss on epoch  60  :  108.341687273 [0.26329443] [0.27053908]
loss on epoch  70  :  64.2979677518 [0.26718548] [0.27817816]
loss on epoch  80  :  40.3065272084 [0.26848248] [0.28091669]
loss on epoch  90  :  27.2785535565 [0.27107653] [0.29792449]
loss on epoch  100  :  19.8817367465 [0.27885863] [0.30513117]
loss on epoch  110  :  15.599368559 [0.29571983] [0.31882387]
loss on epoch  120  :  12.8068944878 [0.29701686] [0.32473335]
loss on epoch  130  :  10.7874289486 [0.30220494] [0.33107525]
loss on epoch  140  :  9.32465522819 [0.30220494] [0.33323723]
loss on epoch  150  :  8.17446415733 [0.30350193] [0.33554339]
loss on epoch  160  :  7.2405699933 [0.30220494] [0.33698472]
loss on epoch  170  :  6.48965020312 [0.30090791] [0.34001154]
loss on epoch  180  :  5.83686744284 [0.30090791] [0.33986738]
loss on epoch  190  :  5.27779822438 [0.30998704] [0.33929086]
loss on epoch  200  :  4.79931404414 [0.31128404] [0.34102046]
loss on epoch  210  :  4.38942148288 [0.30479896] [0.3428942]
loss on epoch  220  :  4.03806058897 [0.30739298] [0.34217355]
loss on epoch  230  :  3.73076107105 [0.30220494] [0.34505621]
loss on epoch  240  :  3.46827077866 [0.30350193] [0.35500145]
loss on epoch  250  :  3.23870945529 [0.30739298] [0.35644278]
loss on epoch  260  :  3.04132128424 [0.31128404] [0.37388295]
loss on epoch  270  :  2.86460503936 [0.30350193] [0.38829634]
loss on epoch  280  :  2.70675845941 [0.29961088] [0.41251081]
loss on epoch  290  :  2.56500498785 [0.30350193] [0.42461804]
loss on epoch  300  :  2.44148617983 [0.29571983] [0.43917555]
loss on epoch  310  :  2.33327854232 [0.28923476] [0.44998559]
loss on epoch  320  :  2.24139836927 [0.28793773] [0.46021909]
loss on epoch  330  :  2.15919722396 [0.28923476] [0.46886712]

In [ ]:
64 64

loss on epoch  0  :  1195.55497798 [0.18028535] [0.16791582]
loss on epoch  10  :  1013.02485487 [0.21660182] [0.21331796]
loss on epoch  20  :  789.418885973 [0.23605707] [0.22081292]
loss on epoch  30  :  567.339089853 [0.2464332] [0.23219948]
loss on epoch  40  :  384.794134069 [0.24124514] [0.23003748]
loss on epoch  50  :  249.932189235 [0.23735408] [0.24257711]
loss on epoch  60  :  158.152248948 [0.26459143] [0.26664746]
loss on epoch  70  :  99.6646442767 [0.26459143] [0.27385414]
loss on epoch  80  :  64.1486431051 [0.26459143] [0.27875468]
loss on epoch  90  :  43.2139298298 [0.2775616] [0.28797925]
loss on epoch  100  :  30.8514907978 [0.28664073] [0.2959066]
loss on epoch  110  :  23.2330012763 [0.27626458] [0.29922166]
loss on epoch  120  :  18.3880767646 [0.27496758] [0.30859038]
loss on epoch  130  :  15.0871106739 [0.28274968] [0.31233785]
loss on epoch  140  :  12.7901680072 [0.2775616] [0.31925628]
loss on epoch  150  :  11.0702089513 [0.2775616] [0.32732776]
loss on epoch  160  :  9.72318348178 [0.27885863] [0.3326607]
loss on epoch  170  :  8.56857403561 [0.28793773] [0.33453444]
loss on epoch  180  :  7.60758964441 [0.28534371] [0.33611992]
loss on epoch  190  :  6.80784432976 [0.28015563] [0.34102046]
loss on epoch  200  :  6.13846226975 [0.29182878] [0.34577689]
loss on epoch  210  :  5.55261748367 [0.28793773] [0.34505621]
loss on epoch  220  :  5.04354822636 [0.28664073] [0.34664169]
loss on epoch  230  :  4.60903858476 [0.28793773] [0.34808302]
loss on epoch  240  :  4.2378280516 [0.28274968] [0.35197464]
loss on epoch  250  :  3.92214061817 [0.29312581] [0.35586625]
loss on epoch  260  :  3.64353918588 [0.29831389] [0.35975784]
loss on epoch  270  :  3.40245219513 [0.29053178] [0.36307293]
loss on epoch  280  :  3.18737982379 [0.29053178] [0.36638802]
loss on epoch  290  :  2.9966332791 [0.30350193] [0.37157682]
loss on epoch  300  :  2.82588767233 [0.30479896] [0.3805131]
loss on epoch  310  :  2.67088949791 [0.30609599] [0.39132315]
loss on epoch  320  :  2.53125948597 [0.31517509] [0.39737678]
loss on epoch  330  :  2.40231606033 [0.31258106] [0.41193429]
loss on epoch  340  :  2.28232390903 [0.30350193] [0.42663592]
loss on epoch  350  :  2.1780065723 [0.30090791] [0.434275]
loss on epoch  360  :  2.0889446029 [0.29831389] [0.44465265]
loss on epoch  370  :  2.01131260947 [0.30350193] [0.45258]
loss on epoch  380  :  1.93970542815 [0.30998704] [0.4628135]
loss on epoch  390  :  1.87546846326 [0.30739298] [0.49048716]
loss on epoch  400  :  1.81851998082 [0.29442284] [0.50951284]


In [3]:
# BACK-UP


# wide, embedding, co-occurrence and feature interaction
# version4: interaction on individual hidden neurons

class InterNN_fuse_full():
    
    def __init__(self, n_conti, n_disc, n_class,\
                 n_disc_voca, session,n_embedding, n_hidden_list, lr, l2, batch_size):
        
#       build the network graph 
        self.LEARNING_RATE = lr
                
        self.N_CLASS = n_class
        self.N_CONTI = n_conti
        self.N_DISC  = n_disc
        self.N_DISC_VOCA = n_disc_voca
        self.N_EMBED = n_embedding
        self.N_BATCH = batch_size
        self.L2 = l2
        
        self.N_HIDDEN_LAYERS = len(n_hidden_list)
        self.N_TOTAL = self.N_CONTI+ self.N_DISC*self.N_EMBED
        self.N_TOTAL_EMBED = self.N_DISC*self.N_EMBED

        self.sess = session
        
        self.dx = tf.placeholder(tf.int32, [None, self.N_DISC])
        self.cx = tf.placeholder(tf.float32, [None, self.N_CONTI])
        self.y  = tf.placeholder(tf.float32, [None, self.N_CLASS])
        self.lr = tf.placeholder(tf.float32, shape=())
        self.keep_prob = tf.placeholder(tf.float32)
        
#       wide part on categorical features
        self.dx_trans = tf.transpose( self.dx, [1,0] )
    
    
#         for i in range(self.N_DISC):
            
#             with tf.variable_scope("wide"+str(i)):
#                 w= tf.Variable(\
#                 tf.random_uniform([self.N_DISC_VOCA[i], self.N_CLASS],-1.0, 1.0) )
# #                         tf.random_normal([self.N_DISC_VOCA, self.N_CLASS],\
# #                         stddev=math.sqrt(2.0/float(self.N_DISC_VOCA))) )
#                 if i==0:
#                     dx_wsum = tf.nn.embedding_lookup( w, self.dx_trans[i] )
                    
# #                   L2
#                     self.regularizer_wide = tf.nn.l2_loss(w)
                    
#                 else:
#                     dx_wsum = dx_wsum + tf.nn.embedding_lookup( w, self.dx_trans[i] )
                    
# #                   L2  
#                     self.regularizer_wide = self.regularizer_wide + tf.nn.l2_loss(w)
               
    
#       co-occurrence of categorical features
        for i in range(20):
            
            tmpvoca1 = self.N_DISC_VOCA[i] 
            
            for j in range(i+1,20):
                
                tmpvoca2 = self.N_DISC_VOCA[j]
                
                with tf.variable_scope("cooccur"+str(i)+str(j)):
                    
                    
#                   vector approximate of co-occurrence matrix 
                    w1 = tf.Variable(tf.random_normal([self.N_CLASS, tmpvoca1, 1],\
                                            stddev=math.sqrt(2.0/float( tmpvoca1 ))) )
                    
                    w2 = tf.Variable(tf.random_normal([self.N_CLASS, 1, tmpvoca2],\
                                            stddev=math.sqrt(2.0/float( tmpvoca2 ))) )
            
                    b  = tf.Variable(tf.zeros( [ self.N_CLASS ] ))
                    
                    cooc_mat = []
                    for k in range(self.N_CLASS):
                        cooc_mat.append( tf.reshape( tf.matmul(w1[k],w2[k]), [-1,1] ) )
                        
#                   n_class by voca1*voca2  
                    cooc_vec = tf.stack(cooc_mat)
                    cooc_vec = tf.reshape(cooc_vec, [self.N_CLASS, -1] )
                    cooc_vec = tf.transpose(cooc_vec,[1,0])
                    
#                   build the idx of co-occurrence features
                    cooc_idx = self.dx_trans[i] * tmpvoca2 + self.dx_trans[j]
    
                    if i==0 and j==1:
#                       co-occurrence weighted sum
                        cooc_wsum  = tf.nn.embedding_lookup( cooc_vec, cooc_idx ) + b
#                       L2 
                        self.regularizer_cooc   = tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2) 
                    else:
                        cooc_wsum += tf.nn.embedding_lookup( cooc_vec, cooc_idx ) + b
#                       L2
                        self.regularizer_cooc += (tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2))
    
#       non-linear on co-occurrence weighted sum
        cooc_wsum = tf.nn.relu( cooc_wsum )
        
        
#       embedding of categorical features        
        tmp_embeded = []                
        for i in range(self.N_DISC):
            
            with tf.variable_scope("embedding"+str(i)):
                w= tf.Variable(\
                        tf.random_normal([self.N_DISC_VOCA[i], self.N_EMBED],\
                        stddev=math.sqrt(2.0/float(self.N_DISC_VOCA[i]))) )
#                     tf.random_uniform([self.N_DISC_VOCA, self.N_EMBED],-1.0, 1.0 )
                              
                tmp_embeded.append( tf.nn.embedding_lookup( w, self.dx_trans[i] ) )
                
        dx_embeded = tf.concat(tmp_embeded, 1)
        x_concate =  tf.concat( [self.cx,dx_embeded], 1 )
        
    
    
#       feature interaction
        expend_cx = tf.reshape( self.cx, [-1,self.N_CONTI,1])  
        expend_cx = tf.transpose( expend_cx, [0,2,1])
        
        for i in range(self.N_DISC):
            
            with tf.variable_scope( "interact"+str(i) ):
                
                w = tf.Variable(\
                tf.random_normal([self.N_DISC_VOCA[i], n_hidden_list[0], self.N_CONTI],\
                        stddev=math.sqrt(2.0/float(self.N_DISC_VOCA[i]))) )
                
                tmp_w = tf.nn.embedding_lookup( w, self.dx_trans[i] )
                
                if i==0:
                    tmp_h = tf.reduce_sum( tmp_w * expend_cx, 2 )
                    
#                   L2
                    self.regularizer_inter = tf.nn.l2_loss(w) 
                    
                else:
                    tmp_h+= tf.reduce_sum( tmp_w * expend_cx, 2 )
                    
#                   L2 
                    self.regularizer_inter += tf.nn.l2_loss(w)


#         tmp_h = []
#         for k in range(n_hidden_list[0]):
            
#             for i in range(self.N_DISC):
                
#                 with tf.variable_scope("interact"+str(i)+"h"+str(k)):
                    
#                     w1 = tf.Variable(\
#                         tf.random_normal([self.N_DISC_VOCA[i], 1],\
#                         stddev=math.sqrt(2.0/float(self.N_DISC_VOCA[i]))) )
                    
#                     w2 = tf.Variable(\
#                         tf.random_normal([1, self.N_CONTI],\
#                         stddev=math.sqrt(2.0/float(self.N_CONTI))) )
                    
#                     w = tf.matmul(w1,w2)
                    
# #                     w = tf.Variable(\
# #                         tf.random_normal([self.N_DISC_VOCA[i], self.N_CONTI],\
# #                         stddev=math.sqrt(2.0/float(self.N_DISC_VOCA[i]))) )

#                     if i==0:
#                         inter_wsum  = tf.nn.embedding_lookup( w, self.dx_trans[i] )
#                     else:
#                         inter_wsum += tf.nn.embedding_lookup( w, self.dx_trans[i] )

#                     if k==0 and i==0:
# #                       L2  
#                         self.regularizer_inter  = tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2) 
#                     else:
# #                       L2
#                         self.regularizer_inter += ( tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2) )
    
#             tmp_h.append( tf.reduce_sum( self.cx * inter_wsum, 1 ) )
            
#         tmp_h = tf.stack( tmp_h )
#         tmp_h = tf.transpose( tmp_h, [1,0] )
        
    
        
#       interaction hidden layers


#         tmp_h = tf.reduce_sum( self.cx * inter_wsum, 2   )
        
#         tmp_h = [ ]
#         for i in range( n_hidden_list[0] ):
#             tmp_h.append( singleCol_h )
        
#         tmp_h = tf.stack( tmp_h )
#         tmp_h = tf.transpose( tmp_h, [1,0])
        
        
        with tf.variable_scope("inter_h0"):
            
            w= tf.Variable(tf.random_normal([self.N_TOTAL, n_hidden_list[0]],\
                        stddev=math.sqrt(2.0/float(self.N_CONTI)))) 
            b= tf.Variable(tf.zeros( [ n_hidden_list[0] ] ))
            
            h_inter = tf.matmul(x_concate, w)
            h_inter = h_inter + tmp_h
            h_inter = tf.nn.relu( tf.add( h_inter, b ) )
            
#           L2   
            self.regularizer = tf.nn.l2_loss(w)
            
#       dropout
#         h_inter = tf.nn.dropout(h_inter, self.keep_prob)
        
        
        
        for i in range(1, self.N_HIDDEN_LAYERS):
            
            with tf.variable_scope("inter_h"+str(i)):
                w = tf.Variable(\
                        tf.random_normal([n_hidden_list[i-1],n_hidden_list[i]],\
                                stddev=math.sqrt(2.0/float(n_hidden_list[i-1])))) 
                b = tf.Variable(tf.zeros( [n_hidden_list[i]] ))
                h_inter = tf.nn.relu( tf.add( tf.matmul(h_inter, w),b) )
        
#               L2
                self.regularizer += tf.nn.l2_loss(w)
        
        
#       output layer  
        h = h_inter
#         h = tf.concat([h_embed, h_inter], 1)
        
        with tf.variable_scope("output"):
            
            w = tf.Variable(tf.random_normal([n_hidden_list[self.N_HIDDEN_LAYERS-1],\
                                             self.N_CLASS],\
            stddev=math.sqrt(2.0/float(n_hidden_list[self.N_HIDDEN_LAYERS-1])))) 
            b = tf.Variable(tf.zeros( [ self.N_CLASS ] ))
            
            h = tf.matmul(h, w)
#             h = tf.add( h, dx_wsum )
            h = tf.add( h, cooc_wsum )
            h = tf.add( h, b )
        
            self.logit = h
            
#           L2  
            self.regularizer += tf.nn.l2_loss(w) 
    
#       overall L2
        self.regularizer += (self.regularizer_cooc + self.regularizer_inter)
#       self.regularizer_wide + 
         
        
#   initialize loss and optimization operations for training
    def train_ini(self):
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(\
                                                    logits=self.logit, labels=self.y) \
                                  + self.L2*self.regularizer )
        self.optimizer = \
        tf.train.AdadeltaOptimizer(learning_rate = self.LEARNING_RATE).minimize(self.cost)
#       tf.train.RMSPropOptimizer(learning_rate = self.LEARNING_RATE).minimize(self.cost)
#         tf.train.AdamOptimizer(learning_rate = self.lr).minimize(self.cost)
#         tf.train.RMSPropOptimizer(learning_rate = self.lr).minimize(self.cost)
#         tf.train.AdadeltaOptimizer(learning_rate = self.lr).minimize(self.cost)
#         tf.train.AdamOptimizer(learning_rate = self.lr).minimize(self.cost)
#         tf.train.GradientDescentOptimizer(learning_rate = self.lr).minimize(self.cost)
        
        self.init = tf.global_variables_initializer()
        sess.run( self.init )
        
#   training on batch of data
    def train_batch(self, dx_batch,cx_batch, y_batch, keep_prob, lr):
        
        _,c = sess.run([self.optimizer,self.cost],\
                       feed_dict={self.dx:dx_batch, \
                                  self.cx:cx_batch, self.y:y_batch,\
                                  self.lr:lr,\
                                  self.keep_prob:keep_prob\
                                 })
        
        return c
        
#   initialize inference         
    def inference_ini(self):
        self.correct_prediction = tf.equal(tf.argmax(self.logit,1), tf.argmax(self.y,1))
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, "float"))
        
#   infer givn testing data    
    def inference(self, dx_test, cx_test, y_test, keep_prob):
        return sess.run([self.accuracy], feed_dict={self.dx:dx_test,\
                                                  self.cx:cx_test, self.y:y_test,\
                                                  self.keep_prob:keep_prob\
                                                   })
        
#   unit_test
    def test(self, dx_test, cx_test, y_test ):
        self.init = tf.global_variables_initializer()
        sess.run( self.init )
        return sess.run( [self.tmp1], \
                 feed_dict={self.dx:dx_test, self.cx:cx_test, self.y:y_test})
        
        

In [ ]:
para_n_epoch = 1200

# tunable parameters

#   representation ability
para_n_hidden_list = [ 16,32,32 ]
#     128,16,8
para_lr = 0.045
para_n_embedding = 2
#   regularization
para_batch_size = 32
para_keep_prob = 0.8
para_l2 = 0.2

# fixed parameters
para_n_class = 8
para_n_disc = 80
para_n_conti = 106
para_n_disc_voca = [8]*para_n_disc

# evaluation parameters
para_eval_byepoch = 10


with tf.Session() as sess:
    
    clf = InterNN_fuse_full( para_n_conti, para_n_disc, para_n_class, \
                      para_n_disc_voca, sess, para_n_embedding, para_n_hidden_list,\
                        para_lr, para_l2, para_batch_size)
    clf.train_ini()
    clf.inference_ini()
    
    para_cur_lr=para_lr
    
    total_cnt= np.shape(cxtrain)[0]
    total_batch = int(total_cnt/para_batch_size)
    
    total_idx=range(total_cnt)
    
    
#   training cycle
    for epoch in range(para_n_epoch):
        
        tmpc=0.0
        
#       shuffle traning instances each epoch  
        np.random.shuffle(total_idx)
    
#       Loop over all batches
        for i in range(total_batch):
            
            batch_idx = total_idx[ i*para_batch_size: (i+1)*para_batch_size ] 
            
            batch_cx = cxtrain[ batch_idx ]
            batch_dx = dxtrain[ batch_idx ]
            batch_y  =  ytrain[ batch_idx ]
            
            tmpc += clf.train_batch( batch_dx, batch_cx, batch_y, para_keep_prob,\
                                     para_cur_lr )
        
        if epoch%para_eval_byepoch != 0:
            continue
    
        tmp_test_acc  = clf.inference(dxtest, cxtest, ytest, para_keep_prob) 
        tmp_train_acc = clf.inference(dxtrain, cxtrain, ytrain, para_keep_prob) 
        
        print "loss on epoch ", epoch, " : ", 1.0*tmpc/total_batch, tmp_test_acc,\
        tmp_train_acc
    
    print "Optimization Finished!"

In [6]:
# for test

w1 = tf.Variable( tf.random_normal([10,2, 3],\
                        stddev=math.sqrt(2.0/float(2))) )
w2 = tf.Variable( tf.random_normal([10,1, 3],\
                        stddev=math.sqrt(2.0/float(2))) )

tmpmul = w1 * w2

w3 = tf.Variable( tf.random_normal( [], stddev=math.sqrt(2.0/float(2))) )

# w2 = tf.Variable( tf.random_normal([10, 5, 6],\
#                         stddev=math.sqrt(2.0/float(2))) )


# idx = tf.constant( [ [1,2],[2,3],[3,4],[9,8] ] )

# tmp = tf.nn.embedding_lookup( [ w2 ], idx )
tmpshape = tf.shape( tmpmul )

with tf.Session() as sess:
    sess.run( tf.global_variables_initializer() )
    print sess.run( [ w3 ], feed_dict={ })
        


[-0.56679606]
